# 1. Projeto: Associação Temporal de Paradas e OPs

Este notebook organiza o pipeline para:

1. Padronizar chaves (preencher `CD_PEDIDO` e `CD_ITEM` a partir de `CD_OP`).
2. Ordenar temporalmente os eventos por máquina, com diagnósticos de monotonicidade.
3. Associar paradas entre ajustes consecutivos.
4. Inferir OP faltante via merge temporal (`merge_asof`) com tolerância.
5. Marcar eventos próximos às trocas de turno.
6. Tratar produções sem **Ajuste** explícito quando o setup for idêntico (ferramental/cores).

---
## Citações do Cliente

> "Exatamente. Não existe uma associação direta entre evento de Parada e evento de Produção. O sistema basicamente conta chapas e registra eventos de parada."

> "Toda vez que um evento é classificado como Ajuste (cód. 1) o operador informa ao sistema qual será a próxima ordem e automaticamente o sistema encerra a ordem anterior; todas as paradas que estiverem dentro do período entre 2 eventos de Ajuste são automaticamente classificadas como pertencentes àquela OP anterior."

> "Acredito que sejam apontamentos indevidos, ou seja, eventos de paradas associados a ordens de produção onde o sistema não conseguiu associar relação, muito provavelmente porque foram classificados erroneamente. Verifique se estes eventos não estão com horários relacionados às trocas de turno (~ 05:00 ; 13:00 ; 21:00)."

# 2. Índice (Table of Contents)

1. Setup e utilitários básicos
2. Dados de entrada e filtros iniciais
3. Seleção e transformação de máquinas
4. Merge base, normalização e diagnósticos temporais
5. Padronização de chaves, escopo e filtros
6. Preparação temporal, associação e inferência de OP
7. Marcação de trocas de turno e diagnósticos rápidos
8. Pipeline integrado, execução final e persistência
9. Produções sem ajuste explícito e auditorias
10. Estatísticas finais, validações e investigações
11. Amostras e utilitários de depuração

# 3. Setup e utilitários básicos

Carrega bibliotecas, define utilitários reaproveitados (strings, datas, diretórios) e configura opções globais do pandas.

In [1]:
import os
import numpy as np
import pandas as pd

pd.set_option("display.max_columns", None)


### 3.1 Funções utilitárias de strings e datas

Agrupa helpers para tratar tokens ausentes e normalizar timestamps antes dos merges temporais.

In [2]:
# Tokens que representam ausência/placeholder
MISSING_TOKENS = {"", "-1", "None", "nan"}


def is_missing_str(s: pd.Series) -> pd.Series:
    s = s.astype("string")
    return s.isna() | s.str.strip().isin(MISSING_TOKENS)


def to_datetime_safe(s: pd.Series) -> pd.Series:
    return pd.to_datetime(s, errors="coerce")


def normalize_datetimes(
    df: pd.DataFrame, cini: str = "DT_INICIO", cfim: str = "DT_FIM"
) -> pd.DataFrame:
    out = df.copy()
    out[cini] = to_datetime_safe(out[cini])
    out[cfim] = to_datetime_safe(out[cfim])
    return out


def ensure_dir(path: str):
    os.makedirs(path, exist_ok=True)


### 3.2 Configuração de logging

Padroniza o registro local para capturar qualquer aviso/erro durante a execução do notebook.

In [3]:
import logging

logging.basicConfig(
    filename="log_nb_refined",
    filemode="a",
)

## 4. Dados de Entrada

Carrega paradas, facas, máquinas e tarefas conforme caminhos do projeto.

In [4]:
df_paradas = pd.read_parquet("../../../data/raw/tb_paradas.parquet")
df_facas = pd.read_parquet("../../../data/raw/tb_facas.parquet")
df_maquinas = pd.read_parquet("../../../data/raw/tb_maquina.parquet")
df_tarefcon = pd.read_parquet("./../../../data/raw/tb_tarefcon.parquet")

### 4.1 Filtro por FL_EXTERNA

Considera apenas paradas relacionadas a produtos (`FL_EXTERNA = 0`) para manter o foco no escopo do cliente.

In [5]:
df_tarefcon = df_tarefcon[df_tarefcon["DT_INICIO"]>"2024-01-01"]

In [6]:
df_tarefcon.shape

(857501, 26)

In [7]:
# df_paradas = df_paradas[df_paradas["FL_EXTERNA"] == 0].reset_index(drop=True)

In [8]:
# df_paradas = df_paradas[df_paradas["FL_EXTERNA"] == 0].reset_index(drop=True)


# 5. Seleção e Transformação de Máquinas

- Filtra `CD_TIPO` em {1, 3}.
- Mantém `ID_GRUPOMAQUINA` em {8, 18, 22, 23}.
- Renomeia `QT_NRDECORES` para `QT_NRDECORES_MAQUINA`.
- Mapeia `CD_TIPO` para texto descritivo.

In [9]:
df_maquinas_fil = df_maquinas[
    df_maquinas["CD_TIPO"].astype("string").isin(["1", "3"])
].copy()
df_maquinas_fil = df_maquinas_fil[
    df_maquinas_fil["ID_GRUPOMAQUINA"].astype("string").isin(["8", "18", "22", "23"])
].copy()
df_maquinas_fil = df_maquinas_fil.rename(
    columns={"QT_NRDECORES": "QT_NRDECORES_MAQUINA"}
)

status_descr = {1: "C/V", 3: "Flexo"}
df_maquinas_fil["TX_TIPO_MAQUINA"] = (
    df_maquinas_fil["CD_TIPO"].astype(int).map(status_descr)
)

In [10]:
df_maquinas_fil.CD_TIPO.value_counts()

CD_TIPO
1    7
3    4
Name: count, dtype: Int64

In [11]:
df_maquinas_fil


,CD_MAQUINA,CD_TIPO,ID_GRUPOMAQUINA,QT_NRDECORES_MAQUINA,TX_TIPO_MAQUINA
10,CUR2,1,23,3.0,C/V
15,DRO1,1,23,2.0,C/V
16,DRO2,1,23,3.0,C/V
17,DRO3,1,22,3.0,C/V
21,EMB3,3,18,4.0,Flexo
26,EVOL1,3,8,4.0,Flexo
35,MID1,3,8,4.0,Flexo
36,MINI1,3,18,4.0,Flexo
47,WAR1,1,22,4.0,C/V
48,WAR2,1,22,4.0,C/V


# 6. Merge Base: Tarefcon x Maquinas

- `df_tarefcon` × `df_maquinas_fil` por `CD_MAQUINA` (inner)
- Enriquecimento com `df_paradas` pelo código `CD_PARADAOUCONV` → `df_paradas.CD_PARADA` (left)
- Reorganização de colunas e normalização de datas

In [12]:
df_merge = df_tarefcon.merge(
    df_maquinas_fil[
        ["CD_MAQUINA", "QT_NRDECORES_MAQUINA", "ID_GRUPOMAQUINA", "TX_TIPO_MAQUINA"]
    ],
    on="CD_MAQUINA",
    how="inner",
).merge(
    df_paradas[["CD_PARADA", "TX_DESCRICAO", "FL_EXTERNA", "FL_USADACONVERSAO"]],
    left_on="CD_PARADAOUCONV",
    right_on="CD_PARADA",
    how="left",
)
df_merge = df_merge[sorted(df_merge.columns)].copy()
df_merge = normalize_datetimes(df_merge, cini="DT_INICIO", cfim="DT_FIM")

In [13]:
df_merge.head()

,CD_FACA,CD_ITEM,CD_MAQUINA,CD_OP,CD_ORIGEM_REGISTRO,CD_PARADA,CD_PARADAOUCONV,CD_PEDIDO,CD_TURMA,CD_USUARIO,DT_FIM,DT_INICIO,DT_TURMA,FL_EXTERNA,FL_PARADA,FL_REPROGRAMACAO,FL_SKIPFEED,FL_USADACONVERSAO,ID_CLIENTE,ID_GRUPOMAQUINA,QT_AJUSTE,QT_ARRANJO,QT_CHAPASALIMENTADAS,QT_NRDECORES_MAQUINA,QT_PRODUZIDA,QT_PROGRAMADA,TX_DESCRICAO,TX_DESC_ORIGEM_REGISTRO,TX_OPONDULADA,TX_TIPO_MAQUINA,VL_DURACAO,VL_DURACAO_PREVISTA,VL_GRAMATURA
0,-1,-1,CUR2,-1,1,102,102,-1,A,223,2024-01-09 13:00:00,2024-01-09 05:00:00,2024-01-09,0.0,1,0,0,1.0,-1,23,0.0,0.0,0.0,3.0,0.0,0.0,FALTA DE INSUMOS,Online CLP,,C/V,480.0,0.0,0.0
1,-1,-1,CUR2,-1,1,102,102,-1,B,223,2024-01-09 21:00:00,2024-01-09 13:00:00,2024-01-09,0.0,1,0,0,1.0,-1,23,0.0,0.0,0.0,3.0,0.0,0.0,FALTA DE INSUMOS,Online CLP,,C/V,480.0,0.0,0.0
2,-1,-1,CUR2,-1,1,102,102,-1,C,223,2024-01-10 05:00:00,2024-01-09 21:00:00,2024-01-09,0.0,1,0,0,1.0,-1,23,0.0,0.0,0.0,3.0,0.0,0.0,FALTA DE INSUMOS,Online CLP,,C/V,480.0,0.0,0.0
3,-1,-1,CUR2,-1,1,102,102,-1,A,223,2024-01-10 13:00:00,2024-01-10 05:00:00,2024-01-10,0.0,1,0,0,1.0,-1,23,0.0,0.0,0.0,3.0,0.0,0.0,FALTA DE INSUMOS,Online CLP,,C/V,480.0,0.0,0.0
4,-1,-1,CUR2,-1,1,102,102,-1,B,223,2024-01-10 21:00:00,2024-01-10 13:00:00,2024-01-10,0.0,1,0,0,1.0,-1,23,0.0,0.0,0.0,3.0,0.0,0.0,FALTA DE INSUMOS,Online CLP,,C/V,480.0,0.0,0.0


In [14]:
# df_merge.to_parquet("tb_merge_base.parquet")

## 6.1 Normalização de Tipos

Garante consistência de tipos para evitar erros em validações posteriores.

In [15]:
# Normalização de tipos para garantir consistência
df_merge["CD_PARADAOUCONV"] = (
    pd.to_numeric(df_merge["CD_PARADAOUCONV"], errors="coerce")
    .fillna(-1)
    .astype("int32")
)

df_merge.head()

,CD_FACA,CD_ITEM,CD_MAQUINA,CD_OP,CD_ORIGEM_REGISTRO,CD_PARADA,CD_PARADAOUCONV,CD_PEDIDO,CD_TURMA,CD_USUARIO,DT_FIM,DT_INICIO,DT_TURMA,FL_EXTERNA,FL_PARADA,FL_REPROGRAMACAO,FL_SKIPFEED,FL_USADACONVERSAO,ID_CLIENTE,ID_GRUPOMAQUINA,QT_AJUSTE,QT_ARRANJO,QT_CHAPASALIMENTADAS,QT_NRDECORES_MAQUINA,QT_PRODUZIDA,QT_PROGRAMADA,TX_DESCRICAO,TX_DESC_ORIGEM_REGISTRO,TX_OPONDULADA,TX_TIPO_MAQUINA,VL_DURACAO,VL_DURACAO_PREVISTA,VL_GRAMATURA
0,-1,-1,CUR2,-1,1,102,102,-1,A,223,2024-01-09 13:00:00,2024-01-09 05:00:00,2024-01-09,0.0,1,0,0,1.0,-1,23,0.0,0.0,0.0,3.0,0.0,0.0,FALTA DE INSUMOS,Online CLP,,C/V,480.0,0.0,0.0
1,-1,-1,CUR2,-1,1,102,102,-1,B,223,2024-01-09 21:00:00,2024-01-09 13:00:00,2024-01-09,0.0,1,0,0,1.0,-1,23,0.0,0.0,0.0,3.0,0.0,0.0,FALTA DE INSUMOS,Online CLP,,C/V,480.0,0.0,0.0
2,-1,-1,CUR2,-1,1,102,102,-1,C,223,2024-01-10 05:00:00,2024-01-09 21:00:00,2024-01-09,0.0,1,0,0,1.0,-1,23,0.0,0.0,0.0,3.0,0.0,0.0,FALTA DE INSUMOS,Online CLP,,C/V,480.0,0.0,0.0
3,-1,-1,CUR2,-1,1,102,102,-1,A,223,2024-01-10 13:00:00,2024-01-10 05:00:00,2024-01-10,0.0,1,0,0,1.0,-1,23,0.0,0.0,0.0,3.0,0.0,0.0,FALTA DE INSUMOS,Online CLP,,C/V,480.0,0.0,0.0
4,-1,-1,CUR2,-1,1,102,102,-1,B,223,2024-01-10 21:00:00,2024-01-10 13:00:00,2024-01-10,0.0,1,0,0,1.0,-1,23,0.0,0.0,0.0,3.0,0.0,0.0,FALTA DE INSUMOS,Online CLP,,C/V,480.0,0.0,0.0


In [16]:
df_merge.drop(columns="CD_PARADA", inplace=True)

In [17]:
# Tipos string
df_merge["CD_MAQUINA"] = df_merge["CD_MAQUINA"].astype("string")
df_merge["CD_OP"] = df_merge["CD_OP"].astype("string")
df_merge["CD_ITEM"] = df_merge["CD_ITEM"].astype("string")
df_merge["CD_PEDIDO"] = df_merge["CD_PEDIDO"].astype("string")

# Tipos datetime (já garantidos em normalize_datetimes, mas reforçar)
df_merge["DT_INICIO"] = pd.to_datetime(df_merge["DT_INICIO"], errors="coerce")
df_merge["DT_FIM"] = pd.to_datetime(df_merge["DT_FIM"], errors="coerce")

print("=== TIPOS NORMALIZADOS ===")
print(
    df_merge[["CD_PARADAOUCONV", "CD_MAQUINA", "CD_OP", "DT_INICIO", "DT_FIM"]].dtypes
)

=== TIPOS NORMALIZADOS ===
CD_PARADAOUCONV             int32
CD_MAQUINA         string[python]
CD_OP              string[python]
DT_INICIO          datetime64[ns]
DT_FIM             datetime64[ns]
dtype: object


In [18]:
# 1. Reconstruir RIGHT exatamente como a função faz
out = df_merge.copy()

out["DT_INICIO"] = pd.to_datetime(out["DT_INICIO"], errors="coerce")
out["DT_FIM"] = pd.to_datetime(out["DT_FIM"], errors="coerce")
out["CD_MAQUINA"] = out["CD_MAQUINA"].astype("string")
out["CD_OP"] = out["CD_OP"].astype("string")


cod = pd.to_numeric(out["CD_PARADAOUCONV"], errors="coerce").fillna(-1)
out["CD_PARADAOUCONV"] = cod.astype("int32")


mask_parada = out["CD_PARADAOUCONV"].ge(1)
mask_sem_op = out["CD_OP"].astype("string").str.strip().isin(["", "-1", "nan", "None"])

right = out.loc[
    (~mask_sem_op) & out["DT_INICIO"].notna(), ["CD_MAQUINA", "DT_INICIO", "CD_OP"]
]

right = right.dropna(subset=["DT_INICIO"])
right = right.sort_values(["CD_MAQUINA", "DT_INICIO"], kind="mergesort").reset_index(
    drop=True
)

In [19]:
right

,CD_MAQUINA,DT_INICIO,CD_OP
0,CUR2,2024-07-17 13:00:00,669849-1/851681
1,CUR2,2024-07-17 13:00:00,669849-1/851681
2,CUR2,2024-07-17 16:16:30,669849-1/851681
3,CUR2,2024-07-17 16:19:20,669849-1/851681
4,CUR2,2024-07-17 16:24:45,669849-1/851681
...,...,...,...
746011,WAR3,2025-11-15 21:00:00,730700-1/852721
746012,WAR3,2025-11-15 21:10:20,730700-1/852721
746013,WAR3,2025-11-15 21:41:50,730700-1/852721
746014,WAR3,2025-11-15 22:28:15,730700-1/852721


In [20]:
# 2. Detectar a quebra de monotonicidade
bad = right["DT_INICIO"].diff().dt.total_seconds() < 0
bad_idx = bad[bad].index.tolist()[:10]
bad_idx


[23613, 106535, 182043, 275060, 347181, 348778, 419784, 501129, 585318, 661482]

# 7. Padronização de Chaves (OP → Pedido/Item)

Em 96% dos registros que apresentam parada, não há valores para `ID_ITEM` e `ID_PEDIDO`. Como `CD_OP` é `PEDIDO/ITEM`, recuperamos ambos **apenas onde estiver faltando**.

In [21]:
def preencher_pedido_item_de_op(
    df: pd.DataFrame,
    op_col: str = "CD_OP",
    pedido_col: str = "CD_PEDIDO",
    item_col: str = "CD_ITEM",
) -> pd.DataFrame:
    out = df.copy()
    if pedido_col not in out.columns:
        out[pedido_col] = pd.NA
    if item_col not in out.columns:
        out[item_col] = pd.NA

    op_str = (
        out.get(op_col, pd.Series(pd.NA, index=out.index)).astype("string").str.strip()
    )
    op_valida = (~is_missing_str(op_str)) & op_str.str.contains("/", regex=False)
    partes = op_str.where(op_valida).str.split("/", n=1, expand=True)
    if isinstance(partes, pd.DataFrame) and partes.shape[1] == 2:
        ped_from_op = partes[0].astype("string").str.strip()
        item_from_op = partes[1].astype("string").str.strip()
        mask_ped_missing = is_missing_str(out[pedido_col].astype("string"))
        mask_item_missing = is_missing_str(out[item_col].astype("string"))
        out.loc[op_valida & mask_ped_missing, pedido_col] = ped_from_op[
            op_valida & mask_ped_missing
        ]
        out.loc[op_valida & mask_item_missing, item_col] = item_from_op[
            op_valida & mask_item_missing
        ]
    return out


# Aplica o preenchimento de chaves
# df_merge = preencher_pedido_item_de_op(df_merge)

# 8. Escopo e Filtros (remover código 0)

Remove eventos com `CD_PARADAOUCONV == 0` (ex.: esteiras).

In [22]:
# mask_zero = pd.to_numeric(df_merge["CD_PARADAOUCONV"], errors="coerce").fillna(0) == 0
# df_merge = df_merge[~mask_zero].copy()

# 9. Preparação Temporal e Monotonicidade

Cria diagnósticos:
- `BAD_ORDER`: `DT_FIM < DT_INICIO`
- `MONO_BREAK`: quebras de monotonicidade de `DT_INICIO` por máquina

In [23]:
def preparar_tempo_monotonico(
    df: pd.DataFrame,
    col_maquina: str = "CD_MAQUINA",
    col_inicio: str = "DT_INICIO",
    col_fim: str = "DT_FIM",
    corrigir_fim_menor_inicio: bool = False,
) -> pd.DataFrame:
    out = normalize_datetimes(df, cini=col_inicio, cfim=col_fim)
    out[col_maquina] = out[col_maquina].astype("string")

    out = out[~(out[col_inicio].isna() & out[col_fim].isna())].copy()

    out["BAD_ORDER"] = (
        out[col_fim].notna()
        & out[col_inicio].notna()
        & (out[col_fim] < out[col_inicio])
    )

    out = out.sort_values(
        [col_maquina, col_inicio, col_fim], kind="mergesort"
    ).reset_index(drop=True)

    out["MONO_BREAK"] = (
        out.groupby(col_maquina, sort=False)[col_inicio]
        .apply(lambda s: s.diff().dt.total_seconds().fillna(0) < 0)
        .reset_index(level=0, drop=True)
        .astype("int8")
    )

    if corrigir_fim_menor_inicio:
        mask_fix = out["BAD_ORDER"]
        out.loc[mask_fix, col_fim] = out.loc[mask_fix, col_inicio]
        out["BAD_ORDER"] = (
            out[col_fim].notna()
            & out[col_inicio].notna()
            & (out[col_fim] < out[col_inicio])
        )
    return out


# Aplica ordenação e diagnósticos
# df_merge = preparar_tempo_monotonico(df_merge, corrigir_fim_menor_inicio=False)

# 10. Associação entre Ajustes (Paradas → OP anterior)

Regra: paradas entre dois ajustes pertencem à **OP anterior**. O ajuste é `CD_PARADAOUCONV == 1`.

In [24]:
def associar_paradas_entre_ajustes(
    df: pd.DataFrame,
    col_maquina: str = "CD_MAQUINA",
    col_evento: str = "CD_PARADAOUCONV",
    col_op: str = "CD_OP",
    ajuste_code: int = 1,
) -> pd.DataFrame:
    out = df.copy()
    out[col_maquina] = out[col_maquina].astype("string")

    op_atual_por_maq = {}
    for idx, row in out.iterrows():
        maq = row[col_maquina]
        cod = pd.to_numeric(row[col_evento], errors="coerce")
        row_op = row[col_op]

        # Ajuste define a nova OP
        if cod == ajuste_code and not is_missing_str(pd.Series([row_op])).iloc[0]:
            op_atual_por_maq[maq] = row_op
            continue

        # Parada (>=1) sem OP herda a OP atual da máquina
        if (cod >= 1) and is_missing_str(pd.Series([row_op])).iloc[0]:
            if maq in op_atual_por_maq:
                out.at[idx, col_op] = op_atual_por_maq[maq]

    return out


# # Aplica associação entre ajustes
# df_merge = df_merge.sort_values(["CD_MAQUINA", "DT_INICIO"], kind="mergesort").reset_index(drop=True)
# df_merge = associar_paradas_entre_ajustes(df_merge)
# df_merge = preencher_pedido_item_de_op(df_merge)

# 11. Inferência via Merge Temporal (asof)

Regra: paradas sem `CD_OP` → casar `DT_FIM` da parada com próximo `DT_INICIO` com `CD_OP` **na mesma máquina** dentro de tolerância (ex.: 30 min).

Para evitar `ValueError: left keys must be sorted`, o merge é feito **por máquina**, ordenando as chaves em cada grupo.

In [25]:
def inferir_op_asof(df: pd.DataFrame, tolerancia_min: int = 30) -> pd.DataFrame:
    out = normalize_datetimes(df)
    out["__IDX__"] = np.arange(len(out))

    cod = pd.to_numeric(out["CD_PARADAOUCONV"], errors="coerce").fillna(0)
    mask_parada = cod >= 1
    mask_sem_op = is_missing_str(out["CD_OP"])

    left_all = out.loc[
        mask_parada & mask_sem_op & out["DT_FIM"].notna(),
        ["__IDX__", "CD_MAQUINA", "DT_FIM"],
    ].copy()
    left_all = left_all.rename(columns={"DT_FIM": "KEY_TIME"})
    right_all = out.loc[
        (~mask_sem_op) & out["DT_INICIO"].notna(), ["CD_MAQUINA", "DT_INICIO", "CD_OP"]
    ].copy()

    # tipagem consistente
    left_all["CD_MAQUINA"] = left_all["CD_MAQUINA"].astype("string")
    right_all["CD_MAQUINA"] = right_all["CD_MAQUINA"].astype("string")

    updates = []
    tol = pd.Timedelta(minutes=tolerancia_min)

    for maq, left in left_all.groupby("CD_MAQUINA", sort=False):
        right = right_all[right_all["CD_MAQUINA"] == maq]
        if left.empty or right.empty:
            continue

        left = left.sort_values(["KEY_TIME"], kind="mergesort").reset_index(drop=True)
        right = right.sort_values(["DT_INICIO"], kind="mergesort").reset_index(
            drop=True
        )

        matched = pd.merge_asof(
            left,
            right,
            left_on="KEY_TIME",
            right_on="DT_INICIO",
            direction="forward",
            tolerance=tol,
        )

        ok = matched[matched["CD_OP"].notna()]
        if not ok.empty:
            updates.extend(ok[["__IDX__", "CD_OP"]].itertuples(index=False, name=None))

    for idx, op in updates:
        out.loc[out["__IDX__"] == idx, "CD_OP"] = op

    out = out.drop(columns=["__IDX__"])
    out = preencher_pedido_item_de_op(out)
    return out


# Aplica merge temporal
# df_merge = inferir_op_asof(df_merge, tolerancia_min=30)

# 12. Marcação de Troca de Turno (05:00, 13:00, 21:00)

Citação do cliente: verificar eventos próximos às trocas de turno. Cria colunas:
- `NEAR_SHIFT_INICIO`
- `NEAR_SHIFT_FIM`

In [26]:
def marcar_troca_turno(df: pd.DataFrame, janela_min: int = 30) -> pd.DataFrame:
    out = df.copy()
    out["DT_INICIO"] = to_datetime_safe(out["DT_INICIO"])
    out["DT_FIM"] = to_datetime_safe(out["DT_FIM"])

    def near_shift(series: pd.Series) -> pd.Series:
        series = to_datetime_safe(series)
        mins = series.dt.hour * 60 + series.dt.minute
        anchors = [5 * 60, 13 * 60, 21 * 60]
        deltas = [
            (mins - a).abs().where((mins - a).abs() <= 720, 1440 - (mins - a).abs())
            for a in anchors
        ]
        dmin = pd.concat(deltas, axis=1).min(axis=1)
        return (dmin <= janela_min).astype("int8")

    out["NEAR_SHIFT_INICIO"] = near_shift(out["DT_INICIO"])
    out["NEAR_SHIFT_FIM"] = near_shift(out["DT_FIM"])
    return out


# Marca trocas de turno
# df_merge = marcar_troca_turno(df_merge, janela_min=30)

# 13. Diagnósticos Rápidos

- Amostra de paradas (>=1) mostrando OP e proximidade de turno.
- Estatísticas básicas de flags temporais.

In [27]:
# amostra_paradas = df_merge[pd.to_numeric(df_merge["CD_PARADAOUCONV"], errors="coerce").fillna(0) >= 1][
#     ["CD_MAQUINA", "DT_INICIO", "DT_FIM", "CD_PARADAOUCONV", "CD_OP", "CD_PEDIDO", "CD_ITEM", "NEAR_SHIFT_INICIO", "NEAR_SHIFT_FIM"]
# ].sample(20, random_state=42) if df_merge.shape[0] >= 20 else df_merge

# stats_flags = {
#     "total_linhas": int(df_merge.shape[0]),
#     "bad_order": int(df_merge.get("BAD_ORDER", pd.Series([False]*len(df_merge))).sum()),
#     "mono_break": int(df_merge.get("MONO_BREAK", pd.Series([0]*len(df_merge))).sum()),
#     "near_shift_inicio": int(df_merge.get("NEAR_SHIFT_INICIO", pd.Series([0]*len(df_merge))).sum()),
#     "near_shift_fim": int(df_merge.get("NEAR_SHIFT_FIM", pd.Series([0]*len(df_merge))).sum()),
# }

# 14. Pipeline Integrado

Executa todas as etapas na ordem recomendada.

In [28]:
def pipeline_paradas_ops(
    df: pd.DataFrame, tolerancia_min: int = 30, janela_turno_min: int = 30
) -> pd.DataFrame:
    """Executa limpeza, associações e inferências chave em uma única chamada."""
    out = df.copy()

    # Normaliza tipos principais
    out = normalize_datetimes(out, cini="DT_INICIO", cfim="DT_FIM")
    out["CD_MAQUINA"] = out["CD_MAQUINA"].astype("string")
    out["CD_OP"] = out["CD_OP"].astype("string")
    out["CD_PARADAOUCONV"] = (
        pd.to_numeric(out["CD_PARADAOUCONV"], errors="coerce")
        .fillna(-1)
        .astype("int32")
    )

    # Preenche chaves e ordena cronologicamente
    out = preencher_pedido_item_de_op(out)
    out = preparar_tempo_monotonico(out, corrigir_fim_menor_inicio=False)

    # Regras de associação determinística seguidas da inferência temporal
    out = associar_paradas_entre_ajustes(out)
    out = inferir_op_asof(out, tolerancia_min=tolerancia_min)

    # Flags finais (turnos) para apoiar diagnósticos
    out = marcar_troca_turno(out, janela_min=janela_turno_min)
    return out


# 15. Execução Final (opcional)

Descomente para executar tudo de ponta a ponta, caso ainda não tenha rodado as etapas acima.

In [29]:
# df_merge = pipeline_paradas_ops(df_merge, tolerancia_min=30, janela_turno_min=30)

### 15.1 Prévia do dataframe processado

Garante uma visão rápida do resultado após aplicar o pipeline completo.

In [30]:
# df_merge.head()


### 15.2 Salvar checkpoint intermediário

Exporta o dataframe consolidado para reutilização em análises posteriores sem rerodar todo o fluxo.

In [31]:
# df_merge.to_parquet("df_merge_process.parquet")

In [32]:
df_merge = pd.read_parquet("df_merge_process.parquet")

# 16. Produções Sem Ajuste Explícito (setup idêntico)

Quando duas OPs consecutivas **não** possuem um Ajuste explícito (cód. 1) entre elas, **mas** o setup é idêntico, não devemos penalizar a ausência de ajuste.

**Critério de setup idêntico (ajustável):**
- Mesmo `CD_FACA` **e**
- Mesma `QT_NRDECORES_MAQUINA` (ou coluna equivalente de cores)

Resultado:
- Cria `SETUP_SAME` por linha (comparando com linha anterior da mesma máquina)
- Cria `AUSENCIA_AJUSTE_PERMITIDA` quando há troca de OP sem evento 1 entre os blocos, mas `SETUP_SAME == 1`

In [33]:
import pandas as pd


def marcar_setup_identico(
    df: pd.DataFrame,
    col_maquina: str = "CD_MAQUINA",
    col_op: str = "CD_OP",
    col_faca: str = "CD_FACA",
    col_cores: str = "QT_NRDECORES_MAQUINA",
) -> pd.DataFrame:
    """
    Marca SETUP_SAME = 1 quando, dentro da mesma máquina, o registro atual tem
    mesma faca e mesma qtde de cores do registro imediatamente anterior.
    - Trata NA/<NA>/''/'-1' como "sem valor".
    - Preenche NAs de comparação (por causa do shift) com False antes de astype.
    """
    out = df.copy()

    out[col_maquina] = out[col_maquina].astype("string")

    faca = out.get(col_faca)
    if faca is None:
        raise KeyError(f"Coluna {col_faca} não encontrada.")
    faca = faca.astype("string").str.strip()
    faca = faca.mask(faca.isna() | faca.isin(["", "-1", "nan", "None"]), pd.NA)

    # Cores como número (se vier string, converte)
    cores = pd.to_numeric(out.get(col_cores), errors="coerce")

    # Comparação com a linha anterior dentro da mesma máquina
    same_faca = faca.groupby(out[col_maquina], sort=False).transform(
        lambda s: s.eq(s.shift(1))
    )
    same_cores = cores.groupby(out[col_maquina], sort=False).transform(
        lambda s: s.eq(s.shift(1))
    )

    # NAs (primeira linha de cada grupo e/ou valores faltantes) -> False
    mask = same_faca.fillna(False) & same_cores.fillna(False)

    out["SETUP_SAME"] = mask.astype("int8")
    return out


def permitir_ausencia_ajuste_sem_penalizar(
    df: pd.DataFrame,
    col_maquina: str = "CD_MAQUINA",
    col_codigo: str = "CD_PARADAOUCONV",
    col_op: str = "CD_OP",
    col_flag: str = "OK_SEM_AJUSTE",
) -> pd.DataFrame:
    """
    Marca OK_SEM_AJUSTE = 1 quando há troca de OP entre linhas consecutivas
    na mesma máquina e SETUP_SAME == 1 (mesma faca e mesmas cores).
    A ideia é permitir a ausência de um evento explícito de Ajuste (cód. 1)
    sem penalizar a troca, se o setup é idêntico.

    Observações:
    - Não forçamos que *não* haja código 1 entre as linhas (checar isso requer varredura
    - inter-registros). Este marcador é a condição necessária (setup idêntico).
    - Convertendo CD_PARADAOUCONV para numérico "on the fly" quando necessário.
    """
    out = df.copy()

    # Garantias de tipo
    out[col_maquina] = out[col_maquina].astype("string")

    # Precisamos de SETUP_SAME antes
    if "SETUP_SAME" not in out.columns:
        raise KeyError(
            "Coluna 'SETUP_SAME' ausente. Execute marcar_setup_identico antes."
        )

    # OPs como string tratada
    op = out[col_op].astype("string").str.strip()
    op = op.mask(op.isna() | op.isin(["", "-1", "nan", "None"]), pd.NA)

    # Troca de OP entre linhas consecutivas da MESMA máquina
    same_mac = out[col_maquina].eq(out[col_maquina].shift(1))
    op_changed = op.ne(op.shift(1))

    # Setup idêntico (já robusto a NA)
    setup_same = out["SETUP_SAME"].fillna(0).astype("int8").eq(1)

    # Sinaliza como “ok sem ajuste explícito”
    out[col_flag] = ((same_mac) & (op_changed) & (setup_same)).astype("int8")

    return out


> BAD_ORDER – marca 1 quando o término registrado (DT_FIM) fica antes do início (DT_INICIO) no mesmo evento; serve para detectar timestamps invertidos ou registros incompletos antes de ordenar cronologicamente 

>MONO_BREAK – vale 1 sempre que, dentro da mesma máquina, o DT_INICIO retrocede em relação ao registro anterior (quebra de monotonicidade). É calculado após ordenar por máquina/início/fim e denuncia problemas de ordenação temporal que inviabilizam o merge_asof

> NEAR_SHIFT_INICIO / NEAR_SHIFT_FIM – flags de proximidade a trocas de turno (horários âncora 05h, 13h, 21h). O helper mede a distância em minutos do início/fim até esses horários e marca 1 quando está dentro da janela definida (30 min por padrão); usados para diagnósticos sobre apontamentos em troca de turno

In [34]:
df_merge[df_merge["BAD_ORDER"] == "1"]

,CD_FACA,CD_ITEM,CD_MAQUINA,CD_OP,CD_ORIGEM_REGISTRO,CD_PARADAOUCONV,CD_PEDIDO,CD_TURMA,CD_USUARIO,DT_FIM,DT_INICIO,DT_TURMA,FL_EXTERNA,FL_PARADA,FL_REPROGRAMACAO,FL_SKIPFEED,FL_USADACONVERSAO,ID_CLIENTE,ID_GRUPOMAQUINA,QT_AJUSTE,QT_ARRANJO,QT_CHAPASALIMENTADAS,QT_NRDECORES_MAQUINA,QT_PRODUZIDA,QT_PROGRAMADA,TX_DESCRICAO,TX_DESC_ORIGEM_REGISTRO,TX_OPONDULADA,TX_TIPO_MAQUINA,VL_DURACAO,VL_DURACAO_PREVISTA,VL_GRAMATURA,BAD_ORDER,MONO_BREAK,NEAR_SHIFT_INICIO,NEAR_SHIFT_FIM


In [35]:
df_merge[df_merge["MONO_BREAK"] == 1]

,CD_FACA,CD_ITEM,CD_MAQUINA,CD_OP,CD_ORIGEM_REGISTRO,CD_PARADAOUCONV,CD_PEDIDO,CD_TURMA,CD_USUARIO,DT_FIM,DT_INICIO,DT_TURMA,FL_EXTERNA,FL_PARADA,FL_REPROGRAMACAO,FL_SKIPFEED,FL_USADACONVERSAO,ID_CLIENTE,ID_GRUPOMAQUINA,QT_AJUSTE,QT_ARRANJO,QT_CHAPASALIMENTADAS,QT_NRDECORES_MAQUINA,QT_PRODUZIDA,QT_PROGRAMADA,TX_DESCRICAO,TX_DESC_ORIGEM_REGISTRO,TX_OPONDULADA,TX_TIPO_MAQUINA,VL_DURACAO,VL_DURACAO_PREVISTA,VL_GRAMATURA,BAD_ORDER,MONO_BREAK,NEAR_SHIFT_INICIO,NEAR_SHIFT_FIM


In [36]:
df_merge = marcar_setup_identico(
    df_merge, col_faca="CD_FACA", col_cores="QT_NRDECORES_MAQUINA"
)

df_merge = permitir_ausencia_ajuste_sem_penalizar(df_merge)


In [37]:
df_merge.head()


,CD_FACA,CD_ITEM,CD_MAQUINA,CD_OP,CD_ORIGEM_REGISTRO,CD_PARADAOUCONV,CD_PEDIDO,CD_TURMA,CD_USUARIO,DT_FIM,DT_INICIO,DT_TURMA,FL_EXTERNA,FL_PARADA,FL_REPROGRAMACAO,FL_SKIPFEED,FL_USADACONVERSAO,ID_CLIENTE,ID_GRUPOMAQUINA,QT_AJUSTE,QT_ARRANJO,QT_CHAPASALIMENTADAS,QT_NRDECORES_MAQUINA,QT_PRODUZIDA,QT_PROGRAMADA,TX_DESCRICAO,TX_DESC_ORIGEM_REGISTRO,TX_OPONDULADA,TX_TIPO_MAQUINA,VL_DURACAO,VL_DURACAO_PREVISTA,VL_GRAMATURA,BAD_ORDER,MONO_BREAK,NEAR_SHIFT_INICIO,NEAR_SHIFT_FIM,SETUP_SAME,OK_SEM_AJUSTE
0,-1,-1,CUR2,-1,1,138,-1,C,223,2022-01-02 05:00:00,2022-01-01 21:00:05,2022-01-01,1.0,1,0,0,1.0,-1,23,0.0,0.0,0.0,3.0,0.0,0.0,FALTA DE PROGRAMAÇÃO,Online CLP,,C/V,480.0,0.0,0.0,False,0,1,1,0,0
1,-1,-1,CUR2,-1,1,138,-1,A,223,2022-01-02 13:00:05,2022-01-02 05:00:00,2022-01-02,1.0,1,0,0,1.0,-1,23,0.0,0.0,0.0,3.0,0.0,0.0,FALTA DE PROGRAMAÇÃO,Online CLP,,C/V,480.0,0.0,0.0,False,0,1,1,0,0
2,-1,-1,CUR2,-1,1,138,-1,B,223,2022-01-02 21:00:00,2022-01-02 13:00:05,2022-01-02,1.0,1,0,0,1.0,-1,23,0.0,0.0,0.0,3.0,0.0,0.0,FALTA DE PROGRAMAÇÃO,Online CLP,,C/V,480.0,0.0,0.0,False,0,1,1,0,0
3,-1,-1,CUR2,-1,1,138,-1,C,223,2022-01-03 05:00:00,2022-01-02 21:00:00,2022-01-02,1.0,1,0,0,1.0,-1,23,0.0,0.0,0.0,3.0,0.0,0.0,FALTA DE PROGRAMAÇÃO,Online CLP,,C/V,480.0,0.0,0.0,False,0,1,1,0,0
4,-1,-1,CUR2,-1,1,138,-1,A,223,2022-01-03 13:00:00,2022-01-03 05:00:00,2022-01-03,1.0,1,0,0,1.0,-1,23,0.0,0.0,0.0,3.0,0.0,0.0,FALTA DE PROGRAMAÇÃO,Online CLP,,C/V,480.0,0.0,0.0,False,0,1,1,0,0


# 17. Auditorias e Sanity Checks

Funções utilitárias para checar condições típicas de erro ou inconsistência.

In [38]:
import pandas as pd


def _is_missing_op(s: pd.Series) -> pd.Series:
    """
    Identifica valores “faltando OP” em uma série de códigos de OP.

    Considera como ausentes:
    - NaN / <NA>
    - strings vazias ("")
    - placeholders "-1", "nan", "None"

    Parameters
    ----------
    s : pd.Series
        Série com códigos de OP (qualquer tipo).

    Returns
    -------
    pd.Series
        Série booleana, True onde o valor é considerado ausente.
    """
    s = s.astype("string").str.strip()
    return s.isna() | s.isin(["", "-1", "nan", "None"])


def audit_asof_sort(df: pd.DataFrame) -> dict:
    """
    Audita se o dataframe atende aos requisitos de ordenação/limpeza
    para um merge_asof por máquina (by="CD_MAQUINA").

    Faz:
    - Monta LEFT: paradas (CD_PARADAOUCONV >= 1) sem OP, com DT_FIM válido.
    - Monta RIGHT: eventos com OP válida, com DT_INICIO válido.
    - Normaliza tipos de data/hora e máquina.
    - Checa monotonicidade GLOBAL de:
        * KEY_TIME (LEFT)
        * DT_INICIO (RIGHT)
    - Checa monotonicidade por máquina (groupby CD_MAQUINA).
    - Conta NaT em KEY_TIME e DT_INICIO.
    - Localiza o primeiro índice de quebra de monotonicidade
      em KEY_TIME e DT_INICIO (se existir).

    Parameters
    ----------
    df : pd.DataFrame
        Dataframe original com colunas:
        - "CD_MAQUINA", "CD_PARADAOUCONV", "CD_OP",
        - "DT_INICIO", "DT_FIM".

    Returns
    -------
    dict
        Dicionário com:
        - "left_sorted_global" : bool
        - "right_sorted_global": bool
        - "left_nat"           : int
        - "right_nat"          : int
        - "left_break_first_idx"  : int | None
        - "right_break_first_idx" : int | None
        - "by_group" : list[dict] com chaves:
            * "CD_MAQUINA"
            * "left_rows", "right_rows"
            * "left_keys_sorted", "right_keys_sorted"
    """
    tmp = df.copy()

    # Normaliza tipos e máscaras-base
    tmp["DT_INICIO"] = pd.to_datetime(tmp["DT_INICIO"], errors="coerce")
    tmp["DT_FIM"] = pd.to_datetime(tmp["DT_FIM"], errors="coerce")
    cod = pd.to_numeric(tmp["CD_PARADAOUCONV"], errors="coerce")

    mask_parada = cod.ge(1)
    mask_sem_op = _is_missing_op(tmp["CD_OP"])

    # LEFT: paradas sem OP com DT_FIM
    left = (
        tmp.loc[
            mask_parada & mask_sem_op & tmp["DT_FIM"].notna(),
            ["CD_MAQUINA", "DT_FIM"],
        ]
        .rename(columns={"DT_FIM": "KEY_TIME"})
        .copy()
    )
    # RIGHT: eventos com OP válida (DT_INICIO)
    right = tmp.loc[
        (~mask_sem_op) & tmp["DT_INICIO"].notna(),
        ["CD_MAQUINA", "DT_INICIO", "CD_OP"],
    ].copy()

    left["CD_MAQUINA"] = left["CD_MAQUINA"].astype("string")
    right["CD_MAQUINA"] = right["CD_MAQUINA"].astype("string")
    right = right.sort_values(
        ["DT_INICIO", "CD_MAQUINA"], kind="mergesort"
    ).reset_index(drop=True)

    print(right.loc[43240:43260])

    # -------- Monotonicidade GLOBAL --------
    left_sorted_global = left["KEY_TIME"].is_monotonic_increasing
    right_sorted_global = right["DT_INICIO"].is_monotonic_increasing

    # Primeiro índice com quebra (se existir)
    def _first_break_idx(times: pd.Series):
        """
        Retorna o índice do primeiro ponto onde a série temporal
        “anda para trás” (diff < 0). Se não houver quebra, retorna None.
        """
        if times.empty:
            return None
        bad = (times.diff().dt.total_seconds() < 0).fillna(False)
        if bad.any():
            return int(times.index[bad.idxmax()])
        return None

    left_break_first = _first_break_idx(left["KEY_TIME"])
    right_break_first = _first_break_idx(right["DT_INICIO"])

    # -------- Monotonicidade por máquina --------
    by_group = []
    for maq, gleft in left.groupby("CD_MAQUINA", sort=False):
        l_sorted = gleft["KEY_TIME"].is_monotonic_increasing
        gright = right.loc[right["CD_MAQUINA"] == maq]
        r_sorted = gright["DT_INICIO"].is_monotonic_increasing
        by_group.append(
            {
                "CD_MAQUINA": str(maq),
                "left_rows": int(gleft.shape[0]),
                "right_rows": int(gright.shape[0]),
                "left_keys_sorted": bool(l_sorted),
                "right_keys_sorted": bool(r_sorted),
            }
        )

    report = {
        "left_sorted_global": bool(left_sorted_global),
        "right_sorted_global": bool(right_sorted_global),
        "left_nat": int(left["KEY_TIME"].isna().sum()),
        "right_nat": int(right["DT_INICIO"].isna().sum()),
        "left_break_first_idx": left_break_first,
        "right_break_first_idx": right_break_first,
        "by_group": by_group,
    }
    return report


def audit_missing_ops(df: pd.DataFrame) -> dict:
    """
    Resume a quantidade de registros com OP faltante no dataframe.

    Considera como “sem OP”:
    - NaN / <NA>
    - strings vazias
    - placeholders "-1", "nan", "None"

    Parameters
    ----------
    df : pd.DataFrame
        Dataframe com coluna "CD_OP".

    Returns
    -------
    dict
        Dicionário com:
        - "total"       : número total de linhas
        - "faltando_op" : número de linhas sem OP válida
    """
    s = df["CD_OP"].astype("string")
    missing = s.isna() | s.str.strip().isin(["", "-1", "nan", "None"])
    return {"total": int(df.shape[0]), "faltando_op": int(missing.sum())}


In [39]:
asof_sort_report = audit_asof_sort(df_merge)
missing_ops_report = audit_missing_ops(df_merge)

      CD_MAQUINA           DT_INICIO             CD_OP
43240       EMB3 2022-02-10 03:56:50   589496-1/626060
43241       EMB3 2022-02-10 03:56:50   589496-1/626060
43242       MID1 2022-02-10 03:57:45   589933-1/809270
43243       MID1 2022-02-10 03:57:45   589933-1/809270
43244       WAR2 2022-02-10 03:59:45   589359-6/658540
43245       WAR1 2022-02-10 04:01:05   589353-1/693530
43246      MINI1 2022-02-10 04:03:25   586756-8/535410
43247       DRO1 2022-02-10 04:05:45   590333-4/718870
43248       DRO2 2022-02-10 04:09:00   590164-1/515020
43249      MINI1 2022-02-10 04:09:15   586756-8/535410
43250       EMB3 2022-02-10 04:11:20   589496-1/626060
43251       WAR3 2022-02-10 04:13:20  589415-17/710671
43252       WAR3 2022-02-10 04:13:20  589415-17/710671
43253       DRO2 2022-02-10 04:13:35   590164-1/515020
43254       EMB3 2022-02-10 04:15:35   589496-1/626060
43255       DRO3 2022-02-10 04:18:53   590091-6/656561
43256      MINI1 2022-02-10 04:19:25   586756-8/535410
43257     

In [40]:
asof_sort_report

{'left_sorted_global': True,
 'right_sorted_global': True,
 'left_nat': 0,
 'right_nat': 0,
 'left_break_first_idx': None,
 'right_break_first_idx': None,
 'by_group': [{'CD_MAQUINA': 'CUR2',
   'left_rows': 5,
   'right_rows': 43239,
   'left_keys_sorted': True,
   'right_keys_sorted': True}]}

In [41]:
print(missing_ops_report)

{'total': 1493740, 'faltando_op': 20}


In [42]:
import pandas as pd


def _is_missing_op(s: pd.Series) -> pd.Series:
    s = s.astype("string").str.strip()
    return s.isna() | s.eq("") | s.eq("-1") | s.eq("nan") | s.eq("None")


def inferir_op_por_asof(df: pd.DataFrame, tolerancia_min: int = 30) -> pd.DataFrame:
    out = df.copy()

    # ----------------------------------------
    # Normaliza tipos
    # ----------------------------------------
    out["DT_INICIO"] = pd.to_datetime(out["DT_INICIO"], errors="coerce")
    out["DT_FIM"] = pd.to_datetime(out["DT_FIM"], errors="coerce")

    out["CD_MAQUINA"] = out["CD_MAQUINA"].astype("string")
    out["CD_OP"] = out["CD_OP"].astype("string")

    cod = pd.to_numeric(out["CD_PARADAOUCONV"], errors="coerce").fillna(-1)
    out["CD_PARADAOUCONV"] = cod.astype("int32")

    mask_parada = out["CD_PARADAOUCONV"].ge(1)
    mask_sem_op = _is_missing_op(out["CD_OP"])

    # ----------------------------------------
    # LEFT: Paradas sem OP
    # ----------------------------------------
    left = out.loc[
        mask_parada & mask_sem_op & out["DT_FIM"].notna(), ["CD_MAQUINA", "DT_FIM"]
    ].rename(columns={"DT_FIM": "KEY_TIME"})

    # Remove NaT se existir
    left = left.dropna(subset=["KEY_TIME"])

    # SORT GLOBAL CORRETO
    left = left.sort_values(["CD_MAQUINA", "KEY_TIME"], kind="mergesort").reset_index(
        drop=True
    )

    # ----------------------------------------
    # RIGHT: Eventos com OP válida
    # ----------------------------------------
    right = out.loc[
        (~mask_sem_op) & out["DT_INICIO"].notna(), ["CD_MAQUINA", "DT_INICIO", "CD_OP"]
    ]

    right = right.dropna(subset=["DT_INICIO"])

    # SORT GLOBAL CORRETO
    right = right.sort_values(
        ["CD_MAQUINA", "DT_INICIO"], kind="mergesort"
    ).reset_index(drop=True)

    # validação correta
    for maq, g in right.groupby("CD_MAQUINA"):
        if not g["DT_INICIO"].is_monotonic_increasing:
            raise ValueError(f"DT_INICIO não é monotônico para a máquina {maq}.")

    # ----------------------------------------
    # MERGE ASOF — AGORA FUNCIONA
    # ----------------------------------------
    matched = pd.merge_asof(
        left,
        right,
        left_on="KEY_TIME",
        right_on="DT_INICIO",
        by="CD_MAQUINA",
        direction="forward",
        tolerance=pd.Timedelta(minutes=tolerancia_min),
    )

    # ----------------------------------------
    # Grava resultado no DF original
    # ----------------------------------------
    for i, row in matched.iterrows():
        op = row["CD_OP"]
        if pd.isna(op):
            continue

        cond = (
            (out["CD_MAQUINA"] == row["CD_MAQUINA"])
            & (out["DT_FIM"] == row["KEY_TIME"])
            & mask_parada
            & mask_sem_op
        )
        out.loc[cond, "CD_OP"] = op

    return out


In [43]:
df_merge = df_merge.sort_values(["CD_MAQUINA", "DT_INICIO", "DT_FIM"]).reset_index(
    drop=True
)

In [44]:
# df_merge = inferir_op_por_asof(df_merge, tolerancia_min=30)
df_merge = inferir_op_asof(df_merge, tolerancia_min=30)

In [45]:
# FILTRO REMOVIDO: FL_EXTERNA já foi filtrado em df_paradas no início
# Manter este filtro aqui removeria produções válidas (CD_PARADAOUCONV = -1)
# que não têm FL_EXTERNA preenchida

# df_merge = df_merge[df_merge["FL_EXTERNA"]==0].reset_index(drop=True)

print(f"Registros após processamento completo: {df_merge.shape[0]:,}")

Registros após processamento completo: 1,493,740


In [46]:
df_merge[df_merge["CD_OP"] == "-1"].shape[0]

19

In [47]:
df_merge.shape[0]

1493740

In [48]:
df_paradas[df_paradas["CD_PARADA"] == "138"]

,CD_PARADA,FL_DESATIVADA,FL_EXTERNA,FL_USADACONVERSAO,TX_DESCRICAO
38,138,0,1,1,FALTA DE PROGRAMAÇÃO


In [49]:
df_merge[df_merge["CD_OP"] == "585577-4/788942"]

,CD_FACA,CD_ITEM,CD_MAQUINA,CD_OP,CD_ORIGEM_REGISTRO,CD_PARADAOUCONV,CD_PEDIDO,CD_TURMA,CD_USUARIO,DT_FIM,DT_INICIO,DT_TURMA,FL_EXTERNA,FL_PARADA,FL_REPROGRAMACAO,FL_SKIPFEED,FL_USADACONVERSAO,ID_CLIENTE,ID_GRUPOMAQUINA,QT_AJUSTE,QT_ARRANJO,QT_CHAPASALIMENTADAS,QT_NRDECORES_MAQUINA,QT_PRODUZIDA,QT_PROGRAMADA,TX_DESCRICAO,TX_DESC_ORIGEM_REGISTRO,TX_OPONDULADA,TX_TIPO_MAQUINA,VL_DURACAO,VL_DURACAO_PREVISTA,VL_GRAMATURA,BAD_ORDER,MONO_BREAK,NEAR_SHIFT_INICIO,NEAR_SHIFT_FIM,SETUP_SAME,OK_SEM_AJUSTE
4,-1,788942,CUR2,585577-4/788942,1,138,585577-4,A,223,2022-01-03 13:00:00,2022-01-03 05:00:00,2022-01-03,1.0,1,0,0,1.0,-1,23,0.0,0.0,0.0,3.0,0.0,0.0,FALTA DE PROGRAMAÇÃO,Online CLP,,C/V,480.0,0.0,0.0,False,0,1,1,0,0
5,-1,788942,CUR2,585577-4/788942,1,138,585577-4,B,223,2022-01-03 21:00:05,2022-01-03 13:00:00,2022-01-03,1.0,1,0,0,1.0,-1,23,0.0,0.0,0.0,3.0,0.0,0.0,FALTA DE PROGRAMAÇÃO,Online CLP,,C/V,480.0,0.0,0.0,False,0,1,1,0,0
6,-1,788942,CUR2,585577-4/788942,1,138,585577-4,C,223,2022-01-03 22:25:05,2022-01-03 21:00:05,2022-01-03,1.0,1,0,0,1.0,-1,23,0.0,0.0,0.0,3.0,0.0,0.0,FALTA DE PROGRAMAÇÃO,Online CLP,,C/V,85.0,0.0,0.0,False,0,1,0,0,0
7,-1,788942,CUR2,585577-4/788942,1,-1,585577-4,C,80,2022-01-04 00:01:50,2022-01-03 21:00:05,2022-01-03,NaN,0,1,0,NaN,11572,23,0.0,6.0,2059.0,3.0,12000.0,12000.0,<NA>,Online CLP,PRD046120/788942,C/V,181.0,74.0,369.0,False,0,1,0,0,0
8,-1,788942,CUR2,585577-4/788942,1,140,585577-4,C,223,2022-01-03 22:26:30,2022-01-03 22:25:20,2022-01-03,0.0,1,0,0,1.0,-1,23,0.0,0.0,0.0,3.0,0.0,0.0,CRAVAMENTO NA ALIMENTAÇÃO,Online CLP,,C/V,1.0,0.0,0.0,False,0,0,0,0,0
9,-1,788942,CUR2,585577-4/788942,1,140,585577-4,C,223,2022-01-03 22:34:25,2022-01-03 22:26:45,2022-01-03,0.0,1,0,0,1.0,-1,23,0.0,0.0,0.0,3.0,0.0,0.0,CRAVAMENTO NA ALIMENTAÇÃO,Online CLP,,C/V,8.0,0.0,0.0,False,0,0,0,0,0
10,-1,788942,CUR2,585577-4/788942,1,140,585577-4,C,223,2022-01-03 22:39:50,2022-01-03 22:37:25,2022-01-03,0.0,1,0,0,1.0,-1,23,0.0,0.0,0.0,3.0,0.0,0.0,CRAVAMENTO NA ALIMENTAÇÃO,Online CLP,,C/V,2.0,0.0,0.0,False,0,0,0,0,0
11,-1,788942,CUR2,585577-4/788942,1,140,585577-4,C,223,2022-01-03 22:41:35,2022-01-03 22:40:00,2022-01-03,0.0,1,0,0,1.0,-1,23,0.0,0.0,0.0,3.0,0.0,0.0,CRAVAMENTO NA ALIMENTAÇÃO,Online CLP,,C/V,1.0,0.0,0.0,False,0,0,0,0,0
12,-1,788942,CUR2,585577-4/788942,1,140,585577-4,C,223,2022-01-03 22:50:10,2022-01-03 22:42:40,2022-01-03,0.0,1,0,0,1.0,-1,23,0.0,0.0,0.0,3.0,0.0,0.0,CRAVAMENTO NA ALIMENTAÇÃO,Online CLP,,C/V,8.0,0.0,0.0,False,0,0,0,0,0
13,-1,788942,CUR2,585577-4/788942,1,140,585577-4,C,223,2022-01-03 22:57:05,2022-01-03 22:52:30,2022-01-03,0.0,1,0,0,1.0,-1,23,0.0,0.0,0.0,3.0,0.0,0.0,CRAVAMENTO NA ALIMENTAÇÃO,Online CLP,,C/V,5.0,0.0,0.0,False,0,0,0,0,0


# 18. Estatísticas finais e cobertura

Resume a distribuição de paradas/produções com ou sem OP e expõe os principais indicadores de cobertura para validar o processamento.

In [50]:
df_merge["CD_PARADAOUCONV"].info()

<class 'pandas.core.series.Series'>
RangeIndex: 1493740 entries, 0 to 1493739
Series name: CD_PARADAOUCONV
Non-Null Count    Dtype
--------------    -----
1493740 non-null  int32
dtypes: int32(1)
memory usage: 5.7 MB


In [51]:
# Validar cobertura por tipo de evento
mask_parada = df_merge["CD_PARADAOUCONV"].ge(1)
mask_producao = df_merge["CD_PARADAOUCONV"].eq(-1)
mask_sem_op = df_merge["CD_OP"].eq("-1")

stats = {
    "total_registros": len(df_merge),
    "paradas_total": int(mask_parada.sum()),
    "paradas_com_op": int((mask_parada & ~mask_sem_op).sum()),
    "paradas_sem_op": int((mask_parada & mask_sem_op).sum()),
    "producoes_total": int(mask_producao.sum()),
    "producoes_com_op": int((mask_producao & ~mask_sem_op).sum()),
    "producoes_sem_op": int((mask_producao & mask_sem_op).sum()),
}

# Calcular percentuais
if stats["paradas_total"] > 0:
    stats["cobertura_paradas_pct"] = (
        stats["paradas_com_op"] / stats["paradas_total"]
    ) * 100
else:
    stats["cobertura_paradas_pct"] = 0.0

stats["cobertura_total_pct"] = ((~mask_sem_op).sum() / len(df_merge)) * 100

print("=" * 60)
print("ESTATÍSTICAS FINAIS DO PROCESSAMENTO")
print("=" * 60)
for k, v in stats.items():
    if "pct" in k:
        print(f"{k:.<40} {v:>10.4f}%")
    else:
        print(f"{k:.<40} {v:>10,}")
print("=" * 60)

# Analisar casos sem OP (se houver)
total_sem_op = stats["paradas_sem_op"] + stats["producoes_sem_op"]
if total_sem_op > 0:
    print(f"\n⚠️  ATENÇÃO: {total_sem_op} registros sem OP identificada")
    print("\n=== CASOS SEM OP (primeiros 20) ===")
    sem_op_df = df_merge[mask_sem_op][
        [
            "CD_MAQUINA",
            "DT_INICIO",
            "DT_FIM",
            "CD_PARADAOUCONV",
            "TX_DESCRICAO",
            "NEAR_SHIFT_INICIO",
            "NEAR_SHIFT_FIM",
        ]
    ].head(20)
    display(sem_op_df)
else:
    print("\n✅ EXCELENTE: Todos os registros têm OP identificada!")

ESTATÍSTICAS FINAIS DO PROCESSAMENTO
total_registros.........................  1,493,740
paradas_total...........................  1,217,148
paradas_com_op..........................  1,217,144
paradas_sem_op..........................          4
producoes_total.........................    276,403
producoes_com_op........................    276,403
producoes_sem_op........................          0
cobertura_paradas_pct...................    99.9997%
cobertura_total_pct.....................    99.9987%

⚠️  ATENÇÃO: 4 registros sem OP identificada

=== CASOS SEM OP (primeiros 20) ===


,CD_MAQUINA,DT_INICIO,DT_FIM,CD_PARADAOUCONV,TX_DESCRICAO,NEAR_SHIFT_INICIO,NEAR_SHIFT_FIM
0,CUR2,2022-01-01 21:00:05,2022-01-02 05:00:00,138,FALTA DE PROGRAMAÇÃO,1,1
1,CUR2,2022-01-02 05:00:00,2022-01-02 13:00:05,138,FALTA DE PROGRAMAÇÃO,1,1
2,CUR2,2022-01-02 13:00:05,2022-01-02 21:00:00,138,FALTA DE PROGRAMAÇÃO,1,1
3,CUR2,2022-01-02 21:00:00,2022-01-03 05:00:00,138,FALTA DE PROGRAMAÇÃO,1,1
688529,EVOL1,2025-10-18 14:00:32,2025-10-18 14:09:09,0,<NA>,0,0
689148,EVOL1,2025-10-29 03:12:53,2025-10-29 03:45:04,0,<NA>,0,0
689206,EVOL1,2025-10-30 00:16:35,2025-10-30 00:27:25,0,<NA>,0,0
689231,EVOL1,2025-10-30 09:16:44,2025-10-30 09:23:50,0,<NA>,0,0
689248,EVOL1,2025-10-30 15:02:51,2025-10-30 15:19:24,0,<NA>,0,0
689268,EVOL1,2025-10-30 22:09:43,2025-10-31 01:13:56,0,<NA>,0,0


# 19. Validações de lógica de negócio

Consolida checagens alinhadas aos requisitos do cliente (monotonicidade, apontamentos em turnos, presença de OP em produções e ajustes).

In [ ]:
"""
Validações de Lógica de Negócio
================================

Segundo a problemática do cliente:
1. "Paradas entre ajustes pertencem à OP anterior"
2. "Sistema encerra ordem anterior quando operador informa próxima OP"
3. "Eventos fora do período início~fim de OP são erros ou externos"
4. "Verificar eventos próximos a trocas de turno (05:00, 13:00, 21:00)"
"""

# ============================================================
# VALIDAÇÃO 1: Eventos de produção (CD_PARADAOUCONV = -1)
# devem SEMPRE ter OP
# ============================================================
mask_producao = df_merge["CD_PARADAOUCONV"].eq(-1)
mask_sem_op = df_merge["CD_OP"].eq("-1")

producoes_sem_op = df_merge[mask_producao & mask_sem_op]
print("=" * 70)
print("VALIDAÇÃO 1: Produções devem SEMPRE ter OP")
print("=" * 70)
if len(producoes_sem_op) > 0:
    print(f"❌ FALHA: {len(producoes_sem_op)} produções sem OP!")
    print("\nPrimeiros 10 casos:")
    display(
        producoes_sem_op[
            ["CD_MAQUINA", "DT_INICIO", "DT_FIM", "CD_PARADAOUCONV", "QT_PRODUZIDA"]
        ].head(10)
    )
else:
    print("✅ PASSOU: Todas as produções têm OP identificada")


VALIDAÇÃO 1: Produções devem SEMPRE ter OP
✅ PASSOU: Todas as produções têm OP identificada


In [ ]:
# ============================================================
# VALIDAÇÃO 2: Paradas próximas a trocas de turno
# (possíveis apontamentos indevidos)
# ============================================================
mask_parada = df_merge["CD_PARADAOUCONV"].ge(1)
mask_near_shift = (df_merge["NEAR_SHIFT_INICIO"] == 1) | (
    df_merge["NEAR_SHIFT_FIM"] == 1
)

paradas_turno = df_merge[mask_parada & mask_near_shift]
total_paradas = mask_parada.sum()
pct_turno = (len(paradas_turno) / total_paradas * 100) if total_paradas > 0 else 0

print(f"\n{'=' * 70}")
print("VALIDAÇÃO 2: Paradas próximas a trocas de turno")
print("=" * 70)
print(f"Total de paradas: {total_paradas:,}")
print(f"Paradas perto de turno (±30min): {len(paradas_turno):,} ({pct_turno:.2f}%)")

if pct_turno > 10:
    print(f"⚠️  ATENÇÃO: {pct_turno:.1f}% das paradas ocorrem perto de trocas de turno")
    print("   Isso pode indicar problemas de apontamento")
else:
    print(f"✅ OK: Apenas {pct_turno:.1f}% das paradas perto de trocas de turno")

# ============================================================
# VALIDAÇÃO 3: Ajustes (código 1) devem TER OP
# (operador informa qual será a próxima ordem)
# ============================================================
mask_ajuste = df_merge["CD_PARADAOUCONV"].eq(1)
ajustes_sem_op = df_merge[mask_ajuste & mask_sem_op]

print(f"\n{'=' * 70}")
print("VALIDAÇÃO 3: Ajustes (código 1) devem ter OP informada")
print("=" * 70)
print(f"Total de ajustes: {mask_ajuste.sum():,}")
print(f"Ajustes sem OP: {len(ajustes_sem_op):,}")

if len(ajustes_sem_op) > 0:
    pct_ajustes_falha = len(ajustes_sem_op) / mask_ajuste.sum() * 100
    print(f"❌ FALHA: {pct_ajustes_falha:.2f}% dos ajustes não têm OP!")
    print("\nPrimeiros 10 ajustes sem OP:")
    display(
        ajustes_sem_op[
            ["CD_MAQUINA", "DT_INICIO", "DT_FIM", "TX_DESCRICAO", "NEAR_SHIFT_INICIO"]
        ].head(10)
    )
else:
    print("✅ PASSOU: Todos os ajustes têm OP identificada")

# ============================================================
# VALIDAÇÃO 4: Consistência temporal dentro de cada máquina
# (OP deve estar ordenada temporalmente)
# ============================================================
print(f"\n{'=' * 70}")
print("VALIDAÇÃO 4: Monotonicidade de OP por máquina")
print("=" * 70)

problemas_ordem = []
for maq in df_merge["CD_MAQUINA"].unique():
    df_maq = df_merge[df_merge["CD_MAQUINA"] == maq].copy()
    df_maq = df_maq[~mask_sem_op].sort_values("DT_INICIO")

    # Detectar trocas de OP para trás no tempo
    op_anterior = None
    for idx, row in df_maq.iterrows():
        op_atual = row["CD_OP"]
        dt_atual = row["DT_INICIO"]

        if op_anterior is not None and op_atual != op_anterior:
            # Houve troca de OP - verificar se faz sentido temporal
            # (simplificado - só alerta se OP volta a anterior)
            pass

        op_anterior = op_atual

mono_breaks = df_merge["MONO_BREAK"].sum()
print(f"Quebras de monotonicidade detectadas: {mono_breaks:,}")

if mono_breaks > 0:
    print(f"⚠️  ATENÇÃO: {mono_breaks} quebras de ordem temporal detectadas")
    print("   Pode indicar problemas no timestamp ou apontamentos")
else:
    print("✅ PASSOU: Timestamps monotônicos em todas as máquinas")

# ============================================================
# VALIDAÇÃO 5: Trocas de OP sem ajuste mas com setup diferente
# (deveria ter ajuste mas não tem)
# ============================================================
print(f"\n{'=' * 70}")
print("VALIDAÇÃO 5: Trocas de OP sem ajuste")
print("=" * 70)

# Contar trocas de OP
df_sorted = df_merge.sort_values(["CD_MAQUINA", "DT_INICIO"]).copy()
df_sorted["OP_MUDOU"] = (
    (df_sorted["CD_MAQUINA"] == df_sorted["CD_MAQUINA"].shift(1))
    & (df_sorted["CD_OP"] != df_sorted["CD_OP"].shift(1))
    & (~df_sorted["CD_OP"].eq("-1"))
)

trocas_op = df_sorted["OP_MUDOU"].sum()
trocas_com_setup_same = df_sorted[
    df_sorted["OP_MUDOU"] & (df_sorted["SETUP_SAME"] == 1)
]
trocas_sem_setup_same = df_sorted[
    df_sorted["OP_MUDOU"] & (df_sorted["SETUP_SAME"] == 0)
]

print(f"Total de trocas de OP: {trocas_op:,}")
print(f"  - Com setup idêntico (OK sem ajuste): {len(trocas_com_setup_same):,}")
print(f"  - Com setup diferente (esperado ajuste): {len(trocas_sem_setup_same):,}")

# Verificar se trocas com setup diferente têm ajuste próximo
if len(trocas_sem_setup_same) > 0:
    print("\n  ℹ️  Trocas com setup diferente SEM ajuste explícito podem indicar:")
    print("     - PCP agrupou ordens similares (normal)")
    print("     - Erro de apontamento (verificar)")

# ============================================================
# RESUMO FINAL
# ============================================================
print(f"\n{'=' * 70}")
print("RESUMO DAS VALIDAÇÕES DE LÓGICA")
print("=" * 70)

validacoes_ok = 0
validacoes_total = 5

if len(producoes_sem_op) == 0:
    validacoes_ok += 1
if pct_turno <= 10:
    validacoes_ok += 1
if len(ajustes_sem_op) == 0:
    validacoes_ok += 1
if mono_breaks == 0:
    validacoes_ok += 1
# Validação 5 é informativa, não conta como falha

print(
    f"\nResultado: {validacoes_ok}/{validacoes_total - 1} validações principais passaram"
)

if validacoes_ok == validacoes_total - 1:
    print("\n✅ EXCELENTE: Processamento consistente com regras de negócio!")
elif validacoes_ok >= 2:
    print("\n⚠️  ATENÇÃO: Alguns problemas detectados - revisar casos acima")
else:
    print("\n❌ CRÍTICO: Múltiplos problemas detectados - revisar lógica")


VALIDAÇÃO 2: Paradas próximas a trocas de turno
Total de paradas: 1,217,148
Paradas perto de turno (±30min): 195,403 (16.05%)
⚠️  ATENÇÃO: 16.1% das paradas ocorrem perto de trocas de turno
   Isso pode indicar problemas de apontamento

VALIDAÇÃO 3: Ajustes (código 1) devem ter OP informada
Total de ajustes: 160,104
Ajustes sem OP: 0
✅ PASSOU: Todos os ajustes têm OP identificada

VALIDAÇÃO 4: Monotonicidade de OP por máquina


/tmp/ipykernel_644098/3333833482.py:62: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_maq = df_maq[~mask_sem_op].sort_values("DT_INICIO")
/tmp/ipykernel_644098/3333833482.py:62: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_maq = df_maq[~mask_sem_op].sort_values("DT_INICIO")
/tmp/ipykernel_644098/3333833482.py:62: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_maq = df_maq[~mask_sem_op].sort_values("DT_INICIO")
/tmp/ipykernel_644098/3333833482.py:62: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_maq = df_maq[~mask_sem_op].sort_values("DT_INICIO")
/tmp/ipykernel_644098/3333833482.py:62: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_maq = df_maq[~mask_sem_op].sort_values("DT_INICIO")
/tmp/ipykernel_644098/3333833482.py:62: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_maq = df_maq[~

Quebras de monotonicidade detectadas: 0
✅ PASSOU: Timestamps monotônicos em todas as máquinas

VALIDAÇÃO 5: Trocas de OP sem ajuste
Total de trocas de OP: 252,302
  - Com setup idêntico (OK sem ajuste): 12,402
  - Com setup diferente (esperado ajuste): 239,900

  ℹ️  Trocas com setup diferente SEM ajuste explícito podem indicar:
     - PCP agrupou ordens similares (normal)
     - Erro de apontamento (verificar)

RESUMO DAS VALIDAÇÕES DE LÓGICA

Resultado: 3/4 validações principais passaram

⚠️  ATENÇÃO: Alguns problemas detectados - revisar casos acima


# 20. Investigação aprofundada - Paradas em trocas de turno

Explora em detalhes os tipos, códigos e durações das paradas registradas próximo aos horários de troca de turno para identificar padrões anômalos.

In [52]:
df_merge[df_merge["CD_PARADAOUCONV"].ge(1)].head(20)


,CD_FACA,CD_ITEM,CD_MAQUINA,CD_OP,CD_ORIGEM_REGISTRO,CD_PARADAOUCONV,CD_PEDIDO,CD_TURMA,CD_USUARIO,DT_FIM,DT_INICIO,DT_TURMA,FL_EXTERNA,FL_PARADA,FL_REPROGRAMACAO,FL_SKIPFEED,FL_USADACONVERSAO,ID_CLIENTE,ID_GRUPOMAQUINA,QT_AJUSTE,QT_ARRANJO,QT_CHAPASALIMENTADAS,QT_NRDECORES_MAQUINA,QT_PRODUZIDA,QT_PROGRAMADA,TX_DESCRICAO,TX_DESC_ORIGEM_REGISTRO,TX_OPONDULADA,TX_TIPO_MAQUINA,VL_DURACAO,VL_DURACAO_PREVISTA,VL_GRAMATURA,BAD_ORDER,MONO_BREAK,NEAR_SHIFT_INICIO,NEAR_SHIFT_FIM,SETUP_SAME,OK_SEM_AJUSTE
0,-1,-1,CUR2,-1,1,138,-1,C,223,2022-01-02 05:00:00,2022-01-01 21:00:05,2022-01-01,1.0,1,0,0,1.0,-1,23,0.0,0.0,0.0,3.0,0.0,0.0,FALTA DE PROGRAMAÇÃO,Online CLP,,C/V,480.0,0.0,0.0,False,0,1,1,0,0
1,-1,-1,CUR2,-1,1,138,-1,A,223,2022-01-02 13:00:05,2022-01-02 05:00:00,2022-01-02,1.0,1,0,0,1.0,-1,23,0.0,0.0,0.0,3.0,0.0,0.0,FALTA DE PROGRAMAÇÃO,Online CLP,,C/V,480.0,0.0,0.0,False,0,1,1,0,0
2,-1,-1,CUR2,-1,1,138,-1,B,223,2022-01-02 21:00:00,2022-01-02 13:00:05,2022-01-02,1.0,1,0,0,1.0,-1,23,0.0,0.0,0.0,3.0,0.0,0.0,FALTA DE PROGRAMAÇÃO,Online CLP,,C/V,480.0,0.0,0.0,False,0,1,1,0,0
3,-1,-1,CUR2,-1,1,138,-1,C,223,2022-01-03 05:00:00,2022-01-02 21:00:00,2022-01-02,1.0,1,0,0,1.0,-1,23,0.0,0.0,0.0,3.0,0.0,0.0,FALTA DE PROGRAMAÇÃO,Online CLP,,C/V,480.0,0.0,0.0,False,0,1,1,0,0
4,-1,788942,CUR2,585577-4/788942,1,138,585577-4,A,223,2022-01-03 13:00:00,2022-01-03 05:00:00,2022-01-03,1.0,1,0,0,1.0,-1,23,0.0,0.0,0.0,3.0,0.0,0.0,FALTA DE PROGRAMAÇÃO,Online CLP,,C/V,480.0,0.0,0.0,False,0,1,1,0,0
5,-1,788942,CUR2,585577-4/788942,1,138,585577-4,B,223,2022-01-03 21:00:05,2022-01-03 13:00:00,2022-01-03,1.0,1,0,0,1.0,-1,23,0.0,0.0,0.0,3.0,0.0,0.0,FALTA DE PROGRAMAÇÃO,Online CLP,,C/V,480.0,0.0,0.0,False,0,1,1,0,0
6,-1,788942,CUR2,585577-4/788942,1,138,585577-4,C,223,2022-01-03 22:25:05,2022-01-03 21:00:05,2022-01-03,1.0,1,0,0,1.0,-1,23,0.0,0.0,0.0,3.0,0.0,0.0,FALTA DE PROGRAMAÇÃO,Online CLP,,C/V,85.0,0.0,0.0,False,0,1,0,0,0
8,-1,788942,CUR2,585577-4/788942,1,140,585577-4,C,223,2022-01-03 22:26:30,2022-01-03 22:25:20,2022-01-03,0.0,1,0,0,1.0,-1,23,0.0,0.0,0.0,3.0,0.0,0.0,CRAVAMENTO NA ALIMENTAÇÃO,Online CLP,,C/V,1.0,0.0,0.0,False,0,0,0,0,0
9,-1,788942,CUR2,585577-4/788942,1,140,585577-4,C,223,2022-01-03 22:34:25,2022-01-03 22:26:45,2022-01-03,0.0,1,0,0,1.0,-1,23,0.0,0.0,0.0,3.0,0.0,0.0,CRAVAMENTO NA ALIMENTAÇÃO,Online CLP,,C/V,8.0,0.0,0.0,False,0,0,0,0,0
10,-1,788942,CUR2,585577-4/788942,1,140,585577-4,C,223,2022-01-03 22:39:50,2022-01-03 22:37:25,2022-01-03,0.0,1,0,0,1.0,-1,23,0.0,0.0,0.0,3.0,0.0,0.0,CRAVAMENTO NA ALIMENTAÇÃO,Online CLP,,C/V,2.0,0.0,0.0,False,0,0,0,0,0


In [53]:
# Análise detalhada das paradas próximas a trocas de turno
mask_parada = df_merge["CD_PARADAOUCONV"].ge(1)
mask_near_shift = (df_merge["NEAR_SHIFT_INICIO"] == 1) | (
    df_merge["NEAR_SHIFT_FIM"] == 1
)

paradas_turno = df_merge[mask_parada & mask_near_shift].copy()

print("=" * 80)
print("INVESTIGAÇÃO: Paradas em Trocas de Turno")
print("=" * 80)

# 1. Por tipo de parada
print("\n1. DISTRIBUIÇÃO POR TIPO DE PARADA (Top 15):")
print("-" * 80)
tipo_dist = paradas_turno["TX_DESCRICAO"].value_counts().head(15)
for desc, count in tipo_dist.items():
    pct = count / len(paradas_turno) * 100
    print(f"  {desc:<50} {count:>7,} ({pct:>5.2f}%)")


INVESTIGAÇÃO: Paradas em Trocas de Turno

1. DISTRIBUIÇÃO POR TIPO DE PARADA (Top 15):
--------------------------------------------------------------------------------
  CRAVAMENTO NO EMPILHADOR                            62,030 (31.74%)
  AJUSTE                                              31,317 (16.03%)
  MANUTENÇÃO OPERACIONAL                              23,073 (11.81%)
  CRAVAMENTO NA ALIMENTAÇÃO                           21,743 (11.13%)
  LIMPEZA GERAL                                       14,289 ( 7.31%)
  PROB AMARRADEIRA OPERACIONAL                         8,229 ( 4.21%)
  TROCA DE TURNO - AUT.                                7,294 ( 3.73%)
  PROBLEMA DE ONDULADEIRA                              5,128 ( 2.62%)
  LIMPEZA DO COLEIRO                                   2,434 ( 1.25%)
  FALTA DE PROGRAMAÇÃO                                 2,394 ( 1.23%)
  TROCA/RETÍFICA POLIURETANO                           2,378 ( 1.22%)
  MANUTENÇÃO CORRETIVA MECÂNICA                        2,215 (

In [54]:
# 2. Códigos específicos
print(f"\n2. CÓDIGOS DE PARADA MAIS COMUNS:")
print("-" * 80)
codigo_dist = paradas_turno["CD_PARADAOUCONV"].value_counts().head(10)
for cod, count in codigo_dist.items():
    pct = count / len(paradas_turno) * 100
    # Tentar pegar descrição
    desc_sample = (
        df_merge[df_merge["CD_PARADAOUCONV"] == cod]["TX_DESCRICAO"].iloc[0]
        if len(df_merge[df_merge["CD_PARADAOUCONV"] == cod]) > 0
        else "N/A"
    )
    print(f"  Cód {cod:>3}: {desc_sample:<45} {count:>7,} ({pct:>5.2f}%)")



2. CÓDIGOS DE PARADA MAIS COMUNS:
--------------------------------------------------------------------------------
  Cód 130: CRAVAMENTO NO EMPILHADOR                       62,030 (31.74%)
  Cód   1: AJUSTE                                         31,317 (16.03%)
  Cód 146: MANUTENÇÃO OPERACIONAL                         23,073 (11.81%)
  Cód 140: CRAVAMENTO NA ALIMENTAÇÃO                      21,743 (11.13%)
  Cód 116: LIMPEZA GERAL                                  14,289 ( 7.31%)
  Cód 119: PROB AMARRADEIRA OPERACIONAL                    8,229 ( 4.21%)
  Cód 131: TROCA DE TURNO - AUT.                           7,294 ( 3.73%)
  Cód 135: PROBLEMA DE ONDULADEIRA                         5,128 ( 2.62%)
  Cód 114: LIMPEZA DO COLEIRO                              2,434 ( 1.25%)
  Cód 138: FALTA DE PROGRAMAÇÃO                            2,394 ( 1.23%)


In [55]:
# 3. Distribuição por horário de turno
print(f"\n3. DISTRIBUIÇÃO POR HORÁRIO DE TROCA:")
print("-" * 80)


def classify_shift(dt):
    if pd.isna(dt):
        return "Desconhecido"
    hour = dt.hour
    minute = dt.minute
    time_min = hour * 60 + minute

    # Distâncias para cada turno
    dist_05 = min(abs(time_min - 5 * 60), abs(time_min - (5 * 60 + 1440)))  # 05:00
    dist_13 = min(abs(time_min - 13 * 60), abs(time_min - (13 * 60 + 1440)))  # 13:00
    dist_21 = min(abs(time_min - 21 * 60), abs(time_min - (21 * 60 + 1440)))  # 21:00

    min_dist = min(dist_05, dist_13, dist_21)

    if min_dist == dist_05:
        return "~05:00 (C→A)"
    elif min_dist == dist_13:
        return "~13:00 (A→B)"
    else:
        return "~21:00 (B→C)"


paradas_turno["SHIFT_CLASSIFICADO"] = paradas_turno["DT_INICIO"].apply(classify_shift)
shift_dist = paradas_turno["SHIFT_CLASSIFICADO"].value_counts()

for shift, count in shift_dist.items():
    pct = count / len(paradas_turno) * 100
    print(f"  {shift:<20} {count:>7,} ({pct:>5.2f}%)")



3. DISTRIBUIÇÃO POR HORÁRIO DE TROCA:
--------------------------------------------------------------------------------
  ~13:00 (A→B)          67,080 (34.33%)
  ~05:00 (C→A)          65,117 (33.32%)
  ~21:00 (B→C)          63,206 (32.35%)


In [56]:
# 4. Duração média das paradas por tipo (perto vs longe de turno)
print(f"\n4. DURAÇÃO MÉDIA DAS PARADAS:")
print("-" * 80)

paradas_turno["DURACAO_MIN"] = (
    paradas_turno["DT_FIM"] - paradas_turno["DT_INICIO"]
).dt.total_seconds() / 60

paradas_normais = df_merge[mask_parada & ~mask_near_shift].copy()

paradas_normais["DURACAO_MIN"] = (
    paradas_normais["DT_FIM"] - paradas_normais["DT_INICIO"]
).dt.total_seconds() / 60

duracao_turno = paradas_turno["DURACAO_MIN"].median()
duracao_normal = paradas_normais["DURACAO_MIN"].median()

print(f"  Paradas perto de turno (mediana): {duracao_turno:.1f} minutos")
print(f"  Paradas normais (mediana): {duracao_normal:.1f} minutos")
print(f"  Diferença: {abs(duracao_turno - duracao_normal):.1f} minutos")



4. DURAÇÃO MÉDIA DAS PARADAS:
--------------------------------------------------------------------------------
  Paradas perto de turno (mediana): 6.0 minutos
  Paradas normais (mediana): 2.9 minutos
  Diferença: 3.1 minutos


In [57]:
if duracao_turno > duracao_normal * 2:
    print("  ⚠️  Paradas em troca de turno são significativamente MAIS LONGAS")
elif duracao_turno < duracao_normal * 0.5:
    print(
        "  ⚠️  Paradas em troca de turno são significativamente MAIS CURTAS (suspeito!)"
    )
else:
    print("  ✅ Durações similares - comportamento normal")


  ⚠️  Paradas em troca de turno são significativamente MAIS LONGAS


In [58]:
# 5. Identificar paradas suspeitas (muito curtas ou em horários exatos)
print(f"\n5. PARADAS SUSPEITAS (duração < 1 min OU exatamente em horário de turno):")
print("-" * 80)


# Exatamente em horário de turno (05:00:00, 13:00:00, 21:00:00)
def is_exact_shift(dt):
    if pd.isna(dt):
        return False
    return (dt.hour in [5, 13, 21]) and (dt.minute == 0) and (dt.second <= 5)


paradas_turno["EXATO_TURNO"] = paradas_turno["DT_INICIO"].apply(is_exact_shift)
paradas_suspeitas = paradas_turno[
    (paradas_turno["DURACAO_MIN"] < 1) | (paradas_turno["EXATO_TURNO"])
]

print(f"  Total de paradas suspeitas: {len(paradas_suspeitas):,}")
print(f"    - Duração < 1 min: {(paradas_turno['DURACAO_MIN'] < 1).sum():,}")
print(f"    - Exatamente em 05:00/13:00/21:00: {paradas_turno['EXATO_TURNO'].sum():,}")

if len(paradas_suspeitas) > 0:
    print(f"\n  Tipos de paradas suspeitas (Top 5):")
    for desc, count in paradas_suspeitas["TX_DESCRICAO"].value_counts().head(5).items():
        print(f"    - {desc}: {count:,}")


5. PARADAS SUSPEITAS (duração < 1 min OU exatamente em horário de turno):
--------------------------------------------------------------------------------
  Total de paradas suspeitas: 40,101
    - Duração < 1 min: 16
    - Exatamente em 05:00/13:00/21:00: 40,085

  Tipos de paradas suspeitas (Top 5):
    - MANUTENÇÃO OPERACIONAL: 14,379
    - TROCA DE TURNO - AUT.: 6,510
    - CRAVAMENTO NO EMPILHADOR: 4,703
    - AJUSTE: 4,697
    - CRAVAMENTO NA ALIMENTAÇÃO: 1,737


In [59]:
# 6. Conclusão
print(f"\n{'=' * 80}")
print("CONCLUSÃO:")
print("=" * 80)

pct_suspeitas = (
    (len(paradas_suspeitas) / len(paradas_turno) * 100) if len(paradas_turno) > 0 else 0
)

if pct_suspeitas > 50:
    print(" CRÍTICO: Mais de 50% das paradas em troca de turno são suspeitas!")
    print("   Provável problema de apontamento ou registro automático incorreto.")
elif pct_suspeitas > 20:
    print("ATENÇÃO: Porção significativa de paradas suspeitas em trocas de turno.")
    print("   Recomenda-se revisar procedimentos de apontamento.")
elif tipo_dist.index[0] in [
    "FALTA DE PROGRAMAÇÃO",
    "TROCA DE TURNO - AUT.",
    "REFEIÇÃO",
]:
    print(" NORMAL: Paradas em trocas de turno são majoritariamente operacionais.")
    print("   Tipos esperados: falta de programação, refeição, limpeza, etc.")
else:
    print("  REVISAR: Paradas em trocas de turno têm tipos inesperados.")
    print(f"   Tipo mais comum: {tipo_dist.index[0]}")


CONCLUSÃO:
ATENÇÃO: Porção significativa de paradas suspeitas em trocas de turno.
   Recomenda-se revisar procedimentos de apontamento.


# 21. Persistência e exportação (opcional)

Sugestão de caminhos e comandos para salvar o dataframe final em formatos Parquet/CSV e garantir reprodutibilidade fora do notebook.

In [60]:
# OUTPUT_DIR = "../../../data/processed"
# ensure_dir(OUTPUT_DIR)
# out_parquet = os.path.join(OUTPUT_DIR, "df_paradas_ops.parquet")
# out_csv     = os.path.join(OUTPUT_DIR, "df_paradas_ops.csv")

# Descomente para salvar
# df_merge.to_parquet(out_parquet, index=False)
# df_merge.to_csv(out_csv, index=False, sep=';')

# 22. Amostras de casos suspeitos (função auxiliar)

Função utilitária que coleta paradas com OP válida próximas às janelas de troca de turno para inspeções rápidas.

In [61]:
def amostras_perto_de_turno(df: pd.DataFrame, n: int = 20) -> pd.DataFrame:
    # Paradas (>=1) com OP válida agora e perto de turno
    cod = pd.to_numeric(df["CD_PARADAOUCONV"], errors="coerce").fillna(0)
    mask_parada = cod >= 1
    mask_op_val = ~is_missing_str(df["CD_OP"]) & df["CD_OP"].astype(
        "string"
    ).str.contains("/")
    mask_turno = (df["NEAR_SHIFT_INICIO"] == 1) | (df["NEAR_SHIFT_FIM"] == 1)
    sample = df.loc[
        mask_parada & mask_op_val & mask_turno,
        [
            "CD_MAQUINA",
            "DT_INICIO",
            "DT_FIM",
            "CD_PARADAOUCONV",
            "CD_OP",
            "CD_PEDIDO",
            "CD_ITEM",
            "NEAR_SHIFT_INICIO",
            "NEAR_SHIFT_FIM",
        ],
    ]
    if sample.shape[0] > n:
        return sample.sample(n, random_state=42)
    return sample


# exemplos (opcional):
amostras_turno = amostras_perto_de_turno(df_merge, n=20)
amostras_turno.head()

,CD_MAQUINA,DT_INICIO,DT_FIM,CD_PARADAOUCONV,CD_OP,CD_PEDIDO,CD_ITEM,NEAR_SHIFT_INICIO,NEAR_SHIFT_FIM
922474,MINI1,2024-03-07 21:00:00,2024-03-07 21:07:40,131,657907-1/866770,657907-1,866770,1,1
822754,MID1,2025-06-30 05:27:35,2025-06-30 05:28:50,130,712116-3/870290,712116-3,870290,1,1
660828,EMB3,2025-03-08 13:00:00,2025-03-08 13:05:30,146,701019-1/841530,701019-1,841530,1,1
488903,DRO3,2024-08-27 13:26:50,2024-08-27 13:33:55,146,677240-1/656541,677240-1,656541,1,0
893132,MINI1,2023-06-25 04:35:00,2023-06-25 04:40:00,1,629469-1/698460,629469-1,698460,1,1


# 23. Trilha detalhada por máquina

Auxilia investigações manuais listando, ordenadamente, os eventos de uma máquina específica com os principais indicadores calculados.

In [62]:
def trilha_maquina(df: pd.DataFrame, maquina: str, n: int = 50) -> pd.DataFrame:
    g = df[df["CD_MAQUINA"].astype("string") == str(maquina)].copy()
    g = g.sort_values(["DT_INICIO", "DT_FIM"], kind="mergesort").reset_index(drop=True)
    cols = [
        "CD_MAQUINA",
        "DT_INICIO",
        "DT_FIM",
        "CD_PARADAOUCONV",
        "IS_AJUSTE",
        "SETUP_SAME",
        "AUSENCIA_AJUSTE_PERMITIDA",
        "CD_OP",
        "CD_PEDIDO",
        "CD_ITEM",
        "NEAR_SHIFT_INICIO",
        "NEAR_SHIFT_FIM",
        "BAD_ORDER",
        "MONO_BREAK",
    ]
    cols = [c for c in cols if c in g.columns]
    if g.shape[0] > n:
        return g[cols].head(n)
    return g[cols]


trilha_maquina(df_merge, maquina="DRO3", n=100)

,CD_MAQUINA,DT_INICIO,DT_FIM,CD_PARADAOUCONV,SETUP_SAME,CD_OP,CD_PEDIDO,CD_ITEM,NEAR_SHIFT_INICIO,NEAR_SHIFT_FIM,BAD_ORDER,MONO_BREAK
0,DRO3,2022-01-01 21:00:05,2022-01-01 22:12:25,130,0,588057-1/812150,588057-1,812150,1,0,False,0
1,DRO3,2022-01-01 21:00:05,2022-01-01 23:34:28,-1,0,588057-1/812150,588057-1,812150,1,0,False,0
2,DRO3,2022-01-01 22:12:45,2022-01-01 22:14:30,130,0,588057-1/812150,588057-1,812150,0,0,False,0
3,DRO3,2022-01-01 22:15:10,2022-01-01 22:19:25,140,0,588057-1/812150,588057-1,812150,0,0,False,0
4,DRO3,2022-01-01 22:20:50,2022-01-01 22:28:35,130,0,588057-1/812150,588057-1,812150,0,0,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...
95,DRO3,2022-01-02 11:01:33,2022-01-02 11:56:54,-1,0,587143-2/656421,587143-2,656421,0,0,False,0
96,DRO3,2022-01-02 11:02:15,2022-01-02 11:03:20,140,0,587143-2/656421,587143-2,656421,0,0,False,0
97,DRO3,2022-01-02 11:17:50,2022-01-02 11:20:50,140,0,587143-2/656421,587143-2,656421,0,0,False,0
98,DRO3,2022-01-02 11:23:05,2022-01-02 11:24:30,130,0,587143-2/656421,587143-2,656421,0,0,False,0


# 24. Estatísticas complementares e máscaras globais

Disponibiliza contagens adicionais de OPs e máscaras de recorte para análises exploratórias específicas.

In [63]:
df_merge.CD_OP.value_counts()

CD_OP
656488-1/781561     429
662366-35/691411    376
665189-3/802951     359
587123-10/727221    320
663865-1/660850     306
                   ... 
618026-6/712651       1
617997-5/827311       1
616510-1/670880       1
616506-1/670880       1
645120-8/464551       1
Name: count, Length: 235229, dtype: Int64

In [64]:
df_merge = df_merge[df_merge["CD_OP"] != "-1"]

In [65]:
mask_parada = df_merge["CD_PARADAOUCONV"].ge(1)
mask_producao = df_merge["CD_PARADAOUCONV"].eq(-1)
mask_sem_op = df_merge["CD_OP"].eq("-1")

print(f"Paradas com OP: {(mask_parada & ~mask_sem_op).sum()}")
print(f"Paradas sem OP: {(mask_parada & mask_sem_op).sum()}")
print(f"Produções com OP: {(mask_producao & ~mask_sem_op).sum()}")


Paradas com OP: 1217144
Paradas sem OP: 0
Produções com OP: 276403


# Output 

In [66]:
df_merge


,CD_FACA,CD_ITEM,CD_MAQUINA,CD_OP,CD_ORIGEM_REGISTRO,CD_PARADAOUCONV,CD_PEDIDO,CD_TURMA,CD_USUARIO,DT_FIM,DT_INICIO,DT_TURMA,FL_EXTERNA,FL_PARADA,FL_REPROGRAMACAO,FL_SKIPFEED,FL_USADACONVERSAO,ID_CLIENTE,ID_GRUPOMAQUINA,QT_AJUSTE,QT_ARRANJO,QT_CHAPASALIMENTADAS,QT_NRDECORES_MAQUINA,QT_PRODUZIDA,QT_PROGRAMADA,TX_DESCRICAO,TX_DESC_ORIGEM_REGISTRO,TX_OPONDULADA,TX_TIPO_MAQUINA,VL_DURACAO,VL_DURACAO_PREVISTA,VL_GRAMATURA,BAD_ORDER,MONO_BREAK,NEAR_SHIFT_INICIO,NEAR_SHIFT_FIM,SETUP_SAME,OK_SEM_AJUSTE
4,-1,788942,CUR2,585577-4/788942,1,138,585577-4,A,223,2022-01-03 13:00:00,2022-01-03 05:00:00,2022-01-03,1.0,1,0,0,1.0,-1,23,0.0,0.0,0.0,3.0,0.0,0.0,FALTA DE PROGRAMAÇÃO,Online CLP,,C/V,480.0,0.0,0.0,False,0,1,1,0,0
5,-1,788942,CUR2,585577-4/788942,1,138,585577-4,B,223,2022-01-03 21:00:05,2022-01-03 13:00:00,2022-01-03,1.0,1,0,0,1.0,-1,23,0.0,0.0,0.0,3.0,0.0,0.0,FALTA DE PROGRAMAÇÃO,Online CLP,,C/V,480.0,0.0,0.0,False,0,1,1,0,0
6,-1,788942,CUR2,585577-4/788942,1,138,585577-4,C,223,2022-01-03 22:25:05,2022-01-03 21:00:05,2022-01-03,1.0,1,0,0,1.0,-1,23,0.0,0.0,0.0,3.0,0.0,0.0,FALTA DE PROGRAMAÇÃO,Online CLP,,C/V,85.0,0.0,0.0,False,0,1,0,0,0
7,-1,788942,CUR2,585577-4/788942,1,-1,585577-4,C,80,2022-01-04 00:01:50,2022-01-03 21:00:05,2022-01-03,NaN,0,1,0,NaN,11572,23,0.0,6.0,2059.0,3.0,12000.0,12000.0,<NA>,Online CLP,PRD046120/788942,C/V,181.0,74.0,369.0,False,0,1,0,0,0
8,-1,788942,CUR2,585577-4/788942,1,140,585577-4,C,223,2022-01-03 22:26:30,2022-01-03 22:25:20,2022-01-03,0.0,1,0,0,1.0,-1,23,0.0,0.0,0.0,3.0,0.0,0.0,CRAVAMENTO NA ALIMENTAÇÃO,Online CLP,,C/V,1.0,0.0,0.0,False,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1493735,7501-1,914951,WAR3,729038-1/914951,1,-1,729038-1,B,472,2025-11-05 18:46:30,2025-11-05 18:27:40,2025-11-05,NaN,0,1,0,NaN,9758,22,108.0,4.0,2450.0,4.0,9600.0,9600.0,<NA>,Online CLP,PRD103073/Várias FTs,C/V,19.0,50.0,416.0,False,0,0,0,0,0
1493736,-1,784021,WAR3,729883-1/784021,1,1,729883-1,B,-1,2025-11-05 18:49:25,2025-11-05 18:46:30,2025-11-05,0.0,1,0,0,1.0,-1,22,0.0,0.0,0.0,4.0,0.0,0.0,AJUSTE,Online CLP,,C/V,3.0,0.0,0.0,False,0,0,0,0,0
1493737,7501-1,784021,WAR3,729883-1/784021,1,-1,729883-1,B,472,2025-11-05 19:11:15,2025-11-05 18:46:30,2025-11-05,NaN,0,1,0,NaN,1827,22,84.0,4.0,3555.0,4.0,14200.0,14400.0,<NA>,Online CLP,PRD103073/Várias FTs,C/V,25.0,67.0,416.0,False,0,0,0,0,0
1493738,-1,784021,WAR3,729883-1/784021,1,130,729883-1,B,-1,2025-11-05 18:55:00,2025-11-05 18:53:40,2025-11-05,0.0,1,0,0,1.0,-1,22,0.0,0.0,0.0,4.0,0.0,0.0,CRAVAMENTO NO EMPILHADOR,Online CLP,,C/V,2.0,0.0,0.0,False,0,0,0,0,0


In [67]:
df = df_merge.sort_values(
    ["CD_MAQUINA", "DT_INICIO", "DT_FIM"], kind="mergesort"
).copy()

is_parada = pd.to_numeric(df["CD_PARADAOUCONV"], errors="coerce").fillna(0).ge(1)

same_maq = (
    df["CD_MAQUINA"].astype("string").eq(df["CD_MAQUINA"].shift().astype("string"))
)
same_op = df["CD_OP"].astype("string").eq(df["CD_OP"].shift().astype("string"))
same_cod = df["CD_PARADAOUCONV"].eq(df["CD_PARADAOUCONV"].shift())

cont_parada = (
    is_parada & is_parada.shift(fill_value=False) & same_maq & same_op & same_cod
)
new_block = (~cont_parada) & is_parada

df["PARADA_BLOCK_ID"] = new_block.cumsum()
df.loc[~is_parada, "PARADA_BLOCK_ID"] = pd.NA

# ---- agregação mantendo todas as colunas ----
cols = df.columns.tolist()

agg_dict = {}
for c in cols:
    if c == "DT_INICIO":
        agg_dict[c] = "min"
    elif c == "DT_FIM":
        agg_dict[c] = "max"
    elif c == "VL_DURACAO":
        agg_dict[c] = "sum"  # soma duração das paradas no bloco
    elif c == "PARADA_BLOCK_ID":
        agg_dict[c] = "first"
    else:
        agg_dict[c] = "first"


paradas_agregadas = (
    df[is_parada].groupby("PARADA_BLOCK_ID", as_index=False).agg(agg_dict)
)

# quantidade de registros em cada bloco
counts = df[is_parada].groupby("PARADA_BLOCK_ID").size()
paradas_agregadas["QT_REGISTROS_PARADA"] = paradas_agregadas["PARADA_BLOCK_ID"].map(
    counts
)

# recompor com produções
df_sem_paradas = df[~is_parada].copy()

df_colapsado = (
    pd.concat([df_sem_paradas, paradas_agregadas], ignore_index=True)
    .sort_values(["CD_MAQUINA", "DT_INICIO", "DT_FIM"], kind="mergesort")
    .reset_index(drop=True)
)

In [68]:
cols_drop = [
    "CD_ORIGEM_REGISTRO",
    "CD_USUARIO",
    "DT_TURMA",
    "FL_USADACONVERSAO",
    "ID_CLIENTE",
    "ID_GRUPOMAQUINA",
    "BAD_ORDER",
    "MONO_BREAK",
    "NEAR_SHIFT_INICIO",
    "NEAR_SHIFT_FIM",
    "SETUP_SAME",
    "OK_SEM_AJUSTE",
    "PARADA_BLOCK_ID",
    "QT_NRDECORES_MAQUINA",
    "TX_DESC_ORIGEM_REGISTRO",
    "TX_OPONDULADA",
]


df_colapsado.drop(columns=cols_drop, inplace=True)
df_colapsado = df_colapsado[df_colapsado["FL_EXTERNA"] != 1].drop(columns="FL_EXTERNA")

### DIAGNÓSTICO: Verificar se colapso está correto

Vamos comparar registros antes e depois do colapso para identificar perdas.

In [69]:
print("=" * 90)
print("DIAGNÓSTICO: COMPARAÇÃO ANTES vs DEPOIS DO COLAPSO")
print("=" * 90)

# Pegar o df ANTES do filtro (logo após o processamento completo)
# ATENÇÃO: O problema pode estar aqui!
# Você fez: df_merge = df_merge[df_merge["CD_OP"]!="-1"]
# Isso REMOVE produções e paradas que ainda não têm OP!

# Vamos analisar o que foi perdido
print("\n1. ANÁLISE DO FILTRO CD_OP != '-1':")
print("-" * 90)

# Recarregar df_merge original (antes do filtro)
# df_merge_original = pd.read_parquet("df_merge_process.parquet")  # se você salvou

# Comparar com o atual
print(f"df_merge atual (após filtro): {len(df_merge):,} registros")
# print(f"df_merge original (antes filtro): {len(df_merge_original):,} registros")
# print(f"Perdidos pelo filtro CD_OP != '-1': {len(df_merge_original) - len(df_merge):,}")

print("\n2. ANÁLISE DO DF (base para colapso):")
print("-" * 90)
print(f"df (usado no colapso): {len(df):,} registros")
print(f"  - Paradas (>=1): {is_parada.sum():,}")
print(f"  - Produções (-1): {(~is_parada).sum():,}")

print("\n3. ANÁLISE DO COLAPSO:")
print("-" * 90)
print(f"paradas_agregadas: {len(paradas_agregadas):,} blocos")
print(f"df_sem_paradas: {len(df_sem_paradas):,} registros")
print(f"df_colapsado TOTAL: {len(df_colapsado):,} registros")

print("\n4. BALANÇO DE REGISTROS:")
print("-" * 90)
print(f"ANTES do colapso:")
print(f"  Total df: {len(df):,}")
print(f"  - Paradas: {is_parada.sum():,}")
print(f"  - Produções: {(~is_parada).sum():,}")

print(f"\nDEPOIS do colapso:")
print(f"  Total df_colapsado: {len(df_colapsado):,}")
mask_parada_col = df_colapsado["CD_PARADAOUCONV"].ge(1)
print(f"  - Paradas: {mask_parada_col.sum():,}")
print(f"  - Produções: {(~mask_parada_col).sum():,}")

# Calcular quantos registros de parada foram colapsados
registros_colapsados = is_parada.sum() - mask_parada_col.sum()
print(f"\n  Registros de parada colapsados: {registros_colapsados:,}")

# Verificar se produções foram perdidas
producoes_perdidas = (~is_parada).sum() - (~mask_parada_col).sum()
if producoes_perdidas != 0:
    print(f"  ⚠️  PRODUÇÕES PERDIDAS: {producoes_perdidas:,}")
else:
    print(f"  ✅ Todas as produções preservadas")

print("\n5. EXEMPLO DE BLOCOS COLAPSADOS:")
print("-" * 90)
# Mostrar um exemplo de bloco com múltiplas paradas
blocos_multi = df[is_parada].groupby("PARADA_BLOCK_ID").size()
blocos_grandes = blocos_multi[blocos_multi > 1].head(3)

if len(blocos_grandes) > 0:
    print("Blocos com múltiplas paradas (primeiros 3):")
    for block_id, count in blocos_grandes.items():
        print(f"\n  Bloco {block_id}: {count} paradas consecutivas")
        exemplo = df[df["PARADA_BLOCK_ID"] == block_id][
            [
                "CD_MAQUINA",
                "CD_OP",
                "CD_PARADAOUCONV",
                "DT_INICIO",
                "DT_FIM",
                "VL_DURACAO",
            ]
        ]
        print(exemplo.to_string(index=False))

        # Mostrar o registro colapsado correspondente
        colapsado_ex = df_colapsado[df_colapsado["QT_REGISTROS_PARADA"] == count].head(
            1
        )[
            [
                "CD_MAQUINA",
                "CD_OP",
                "CD_PARADAOUCONV",
                "DT_INICIO",
                "DT_FIM",
                "VL_DURACAO",
                "QT_REGISTROS_PARADA",
            ]
        ]
        print(f"\n  → Colapsado em:")
        print(colapsado_ex.to_string(index=False))
else:
    print("Nenhum bloco com múltiplas paradas encontrado")

print("\n" + "=" * 90)
print("CONCLUSÃO:")
print("=" * 90)

if producoes_perdidas == 0 and registros_colapsados > 0:
    print(f"✅ COLAPSO CORRETO!")
    print(f"   - {registros_colapsados:,} registros de parada foram agrupados")
    print(f"   - Todas as produções foram preservadas")
    print(f"   - Redução total: {len(df) - len(df_colapsado):,} registros")
elif producoes_perdidas > 0:
    print(f"❌ PROBLEMA DETECTADO!")
    print(f"   - {producoes_perdidas:,} PRODUÇÕES FORAM PERDIDAS!")
    print(f"   - Verifique o filtro df_merge = df_merge[df_merge['CD_OP']!='-1']")
else:
    print(f"⚠️  ATENÇÃO: Nenhum colapso foi realizado")
    print(f"   - Não há paradas sequenciais com mesmo código para colapsar")


DIAGNÓSTICO: COMPARAÇÃO ANTES vs DEPOIS DO COLAPSO

1. ANÁLISE DO FILTRO CD_OP != '-1':
------------------------------------------------------------------------------------------
df_merge atual (após filtro): 1,493,721 registros

2. ANÁLISE DO DF (base para colapso):
------------------------------------------------------------------------------------------
df (usado no colapso): 1,493,721 registros
  - Paradas (>=1): 1,217,144
  - Produções (-1): 276,577

3. ANÁLISE DO COLAPSO:
------------------------------------------------------------------------------------------
paradas_agregadas: 873,425 blocos
df_sem_paradas: 276,577 registros
df_colapsado TOTAL: 1,127,280 registros

4. BALANÇO DE REGISTROS:
------------------------------------------------------------------------------------------
ANTES do colapso:
  Total df: 1,493,721
  - Paradas: 1,217,144
  - Produções: 276,577

DEPOIS do colapso:
  Total df_colapsado: 1,127,280
  - Paradas: 850,703
  - Produções: 276,577

  Registros de par

In [70]:
df_colapsado.QT_REGISTROS_PARADA.fillna(0, inplace=True)

/tmp/ipykernel_819729/886713476.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_colapsado.QT_REGISTROS_PARADA.fillna(0, inplace=True)


## Feature Selection and Split

In [71]:
df_colapsado.columns

Index(['CD_FACA', 'CD_ITEM', 'CD_MAQUINA', 'CD_OP', 'CD_PARADAOUCONV',
       'CD_PEDIDO', 'CD_TURMA', 'DT_FIM', 'DT_INICIO', 'FL_PARADA',
       'FL_REPROGRAMACAO', 'FL_SKIPFEED', 'QT_AJUSTE', 'QT_ARRANJO',
       'QT_CHAPASALIMENTADAS', 'QT_PRODUZIDA', 'QT_PROGRAMADA', 'TX_DESCRICAO',
       'TX_TIPO_MAQUINA', 'VL_DURACAO', 'VL_DURACAO_PREVISTA', 'VL_GRAMATURA',
       'QT_REGISTROS_PARADA'],
      dtype='object')

In [72]:
# 1) Separar produção x parada
df_paradas = df_colapsado[df_colapsado["FL_PARADA"] == "1"].copy()
df_producao = df_colapsado[df_colapsado["FL_PARADA"] == "0"].copy()

# 2) Agregados de PARADAS por OP
agg_paradas = df_paradas.groupby("CD_OP", as_index=False).agg(
    VL_DURACAO_PARADAS=("VL_DURACAO", "sum"),  # duração total parado (min)
    QT_PARADAS=("QT_REGISTROS_PARADA", "sum"),  # quantidade de eventos de parada
)

# 3) (Opcional) Agregados de PRODUÇÃO por OP
agg_prod = df_producao.groupby("CD_OP", as_index=False).agg(
    QT_PRODUZIDA=("QT_PRODUZIDA", "sum"),
    QT_CHAPASALIMENTADAS=("QT_CHAPASALIMENTADAS", "sum"),
    VL_DURACAO_PRODUCAO=("VL_DURACAO", "sum"),
    QT_PROGRAMADA=("QT_PROGRAMADA", "max"),
    TX_TIPO_MAQUINA=("TX_TIPO_MAQUINA", "first"),  # mantém para filtro depois
)


# 4) Consolidar em uma linha por CD_OP
df_ops_agg = agg_prod.merge(agg_paradas, on="CD_OP", how="left").fillna(
    {"VL_DURACAO_PARADAS": 0, "QT_PARADAS": 0}
)

# df_ops_agg agora é a “tabela fato” de processo por OP para ser usada adiante


In [73]:
df_ops_agg

,CD_OP,QT_PRODUZIDA,QT_CHAPASALIMENTADAS,VL_DURACAO_PRODUCAO,QT_PROGRAMADA,TX_TIPO_MAQUINA,VL_DURACAO_PARADAS,QT_PARADAS
0,568010-9/797900,12393.0,12488.0,109.0,13100.0,Flexo,41.0,9.0
1,570157-3/797920-2,9600.0,9733.0,100.0,9767.0,Flexo,35.0,6.0
2,573982-1/697050-2,11375.0,11404.0,163.0,11418.0,C/V,22.0,3.0
3,574075-1/719410,2057.0,695.0,31.0,2097.0,C/V,19.0,1.0
4,577288-6/721090,12927.0,3246.0,101.0,12872.0,C/V,35.0,2.0
...,...,...,...,...,...,...,...,...
235169,729824-1/424251,4200.0,2100.0,42.0,4200.0,C/V,7.0,4.0
235170,729882-10/702341,14800.0,4934.0,38.0,13107.0,C/V,7.0,2.0
235171,729883-1/784021,14200.0,3555.0,25.0,14400.0,C/V,17.0,3.0
235172,REP021780/120000,6437.0,7265.0,1446.0,19999.0,C/V,1574.0,78.0


In [76]:
df_final = df_ops_agg.copy()


In [77]:
df_final.to_parquet("df_final_base.parquet")

In [78]:
df_final.TX_TIPO_MAQUINA.value_counts()


TX_TIPO_MAQUINA
C/V      164461
Flexo     70713
Name: count, dtype: int64

### Flexo

In [79]:
df_final_flexo = df_final[df_final["TX_TIPO_MAQUINA"] == "Flexo"].reset_index(drop=True)

In [80]:
df_final_flexo.head()

,CD_OP,QT_PRODUZIDA,QT_CHAPASALIMENTADAS,VL_DURACAO_PRODUCAO,QT_PROGRAMADA,TX_TIPO_MAQUINA,VL_DURACAO_PARADAS,QT_PARADAS
0,568010-9/797900,12393.0,12488.0,109.0,13100.0,Flexo,41.0,9.0
1,570157-3/797920-2,9600.0,9733.0,100.0,9767.0,Flexo,35.0,6.0
2,581953-2/779500,7302.0,7303.0,70.0,7329.0,Flexo,21.0,7.0
3,582104-1/797890,26524.0,26535.0,131.0,25200.0,Flexo,31.0,8.0
4,582104-2/797890,12600.0,12600.0,147.0,12600.0,Flexo,68.0,10.0


> FL_SKIPFEED Se tratam de caixas especiais onde a medida da peça (Largura) é maior que a capacidade da máquina, sendo necessário 2 ciclos para produzir 1 caixa.

In [81]:
df_final_flexo.shape

(70713, 8)

In [82]:
df_final_flexo.to_parquet("../../../data/ml/table_final_flexo.parquet")

In [83]:
df_final_flexo.shape

(70713, 8)

### CV

In [84]:
df_final_cv = df_final[df_final["TX_TIPO_MAQUINA"] == "C/V"].reset_index(drop=True)


In [85]:
df_final_cv

,CD_OP,QT_PRODUZIDA,QT_CHAPASALIMENTADAS,VL_DURACAO_PRODUCAO,QT_PROGRAMADA,TX_TIPO_MAQUINA,VL_DURACAO_PARADAS,QT_PARADAS
0,573982-1/697050-2,11375.0,11404.0,163.0,11418.0,C/V,22.0,3.0
1,574075-1/719410,2057.0,695.0,31.0,2097.0,C/V,19.0,1.0
2,577288-6/721090,12927.0,3246.0,101.0,12872.0,C/V,35.0,2.0
3,577288-8/721090,11355.0,3003.0,29.0,12000.0,C/V,7.0,1.0
4,578237-2/760980,12000.0,3000.0,69.0,12000.0,C/V,39.0,6.0
...,...,...,...,...,...,...,...,...
164456,729824-1/424251,4200.0,2100.0,42.0,4200.0,C/V,7.0,4.0
164457,729882-10/702341,14800.0,4934.0,38.0,13107.0,C/V,7.0,2.0
164458,729883-1/784021,14200.0,3555.0,25.0,14400.0,C/V,17.0,3.0
164459,REP021780/120000,6437.0,7265.0,1446.0,19999.0,C/V,1574.0,78.0


In [86]:
df_final_cv.to_parquet("../../../data/ml/table_final_cv.parquet")
